---


**Data Gathering**

---

In [ ]:
import csv
import datetime as dt
import json
import os
import statistics
import time
import re
import numpy as np
import pandas as pd
import requests
from google.colab import files
from ast import literal_eval
import itertools
pd.set_option('display.max_columns', None)

In [ ]:
def get_request(url, parameters=None):
    """Return json-formatted response of a get request using optional parameters.

    Parameters
    ----------
    url : string
    parameters : {'parameter': 'value'}
        parameters to pass as part of get request

    Returns
    -------
    json_data
        json-formatted response (dict-like)
    """
    try:
        response = requests.get(url=url, params=parameters)
    except SSLError as s:
        print('SSL Error:', s)

        for i in range(5, 0, -1):
            print('\rWaiting... ({})'.format(i), end='')
            time.sleep(1)
        print('\rRetrying.' + ' '*10)

        return get_request(url, parameters)

    if response:
        return response.json()
    else:
        print('No response, waiting 10 seconds...')
        time.sleep(10)
        print('Retrying.')
        return get_request(url, parameters)

In [ ]:
url = "https://steamspy.com/api.php"
parameters = {"request": "all"}
json_data = get_request(url, parameters=parameters)
steam_spy_all = pd.DataFrame.from_dict(json_data, orient='index')
app_list = steam_spy_all[['appid', 'name']].sort_values('appid').reset_index(drop=True)
app_list = pd.read_csv('/content/app_list.csv')
app_list

,appid,name
0,1200,Red Orchestra: Ostfront 41-45
1,1510,Uplink
2,1930,Two Worlds Epic Edition
3,2270,Wolfenstein 3D
4,2290,Final DOOM
...,...,...
995,2218750,Halls of Torment
996,2231380,Tom Clancy's Ghost Recon Breakpoint
997,2231450,Pizza Tower
998,2296990,We Were Here Expeditions: The FriendShip


from matplotlib import pyplot as plt
app_list['appid'].plot(kind='hist', bins=20, title='appid')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['appid']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'appid'}, axis=1)
              .sort_values('appid', ascending=True))
  xs = counted['appid']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = app_list.sort_values('appid', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('appid')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
app_list['appid'].plot(kind='line', figsize=(8, 4), title='appid')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
def get_app_data(start, stop, parser, pause):
    """Return list of app data generated from parser.

    parser : function to handle request
    """
    app_data = []
    for index, row in app_list[start:stop].iterrows():
        print('Current index: {}'.format(index), end='\r')
        appid = row['appid']
        name = row['name']
        data = parser(appid, name)
        app_data.append(data)

        time.sleep(pause) # Untuk digunakan untuk tidak overloading api dengan requests

    return app_data

def process_batches(parser, app_list, download_path, data_filename, index_filename,columns, begin=0, end=-1, batchsize=100, pause=1):
    """Process app data in batches, writing directly to file.
    parser : custom function to format request
    app_list : dataframe of appid and name
    download_path : path to store data
    data_filename : filename to save app data
    index_filename : filename to store highest index written
    columns : column names for file

    Keyword arguments:
    begin : starting index (get from index_filename, default 0)
    end : index to finish (defaults to end of app_list)
    batchsize : number of apps to write in each batch (default 100)
    pause : time to wait after each api request (defualt 1)

    returns: none
    """
    print('Starting at index {}:\n'.format(begin))
    if end == -1:
        end = len(app_list) + 1

    batches = np.arange(begin, end, batchsize)
    batches = np.append(batches, end)

    apps_written = 0
    batch_times = []

    for i in range(len(batches) - 1):
        start_time = time.time()

        start = batches[i]
        stop = batches[i+1]

        app_data = get_app_data(start, stop, parser, pause)

        rel_path = os.path.join(download_path, data_filename)

        with open(rel_path, 'a', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=columns, extrasaction='ignore')

            for j in range(3,0,-1):
                print("\rAbout to write data, don't stop script! ({})".format(j), end='')
                time.sleep(0.5)

            writer.writerows(app_data)
            print('\rExported lines {}-{} to {}.'.format(start, stop-1, data_filename), end=' ')

        apps_written += len(app_data)

        idx_path = os.path.join(download_path, index_filename)

        with open(idx_path, 'w') as f:
            index = stop
            print(index, file=f)

        # logging time taken
        end_time = time.time()
        time_taken = end_time - start_time
        batch_times.append(time_taken)
        mean_time = statistics.mean(batch_times)
        est_remaining = (len(batches) - i - 2) * mean_time
        remaining_td = dt.timedelta(seconds=round(est_remaining))
        time_td = dt.timedelta(seconds=round(time_taken))
        mean_td = dt.timedelta(seconds=round(mean_time))
        print('Batch {} time: {} (avg: {}, remaining: {})'.format(i, time_td, mean_td, remaining_td))
    print('\nProcessing batches complete. {} apps written'.format(apps_written))


In [ ]:
def reset_index(download_path, index_filename):
    """Reset index in file to 0."""
    rel_path = os.path.join(download_path, index_filename)
    with open(rel_path, 'w') as f:
        print(0, file=f)

def get_index(download_path, index_filename):
    """Retrieve index from file, returning 0 if file not found."""
    try:
        rel_path = os.path.join(download_path, index_filename)
        with open(rel_path, 'r') as f:
            index = int(f.readline())
    except FileNotFoundError:
        index = 0
    return index

def prepare_data_file(download_path, filename, index, columns):
    """Create file and write headers if index is 0."""
    if index == 0:
        rel_path = os.path.join(download_path, filename)
        with open(rel_path, 'w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=columns)
            writer.writeheader()

In [ ]:
def parse_steam_request(appid, name):
    """Unique parser to handle data from Steam Store API.

    Returns : json formatted data (dict-like)
    """
    url = "http://store.steampowered.com/api/appdetails/"
    parameters = {"appids": appid}
    json_data = get_request(url, parameters=parameters)
    json_app_data = json_data[str(appid)]

    if json_app_data['success']:
        data = json_app_data['data']
    else:
        data = {'name': name, 'steam_appid': appid}
    return data


# Set file parameters
download_path = '/content'
steam_app_data = 'steam_app_data.csv'
steam_index = 'steam_index.txt'

steam_columns = [
    'type', 'name', 'steam_appid', 'required_age', 'is_free', 'controller_support',
    'dlc', 'detailed_description', 'about_the_game', 'short_description', 'fullgame',
    'supported_languages', 'header_image', 'website', 'pc_requirements', 'mac_requirements',
    'linux_requirements', 'legal_notice', 'drm_notice', 'ext_user_account_notice',
    'developers', 'publishers', 'demos', 'price_overview', 'packages', 'package_groups',
    'platforms', 'metacritic', 'reviews', 'categories', 'genres', 'screenshots',
    'movies', 'recommendations', 'achievements', 'release_date', 'support_info',
    'background', 'content_descriptors'
]

reset_index(download_path, steam_index)
index = get_index(download_path, steam_index)
prepare_data_file(download_path, steam_app_data, index, steam_columns)

process_batches(
    parser=parse_steam_request,
    app_list=app_list,
    download_path=download_path,
    data_filename=steam_app_data,
    index_filename=steam_index,
    columns=steam_columns,
    begin=index,
)

Starting at index 0:

Exported lines 0-99 to steam_app_data.csv. Batch 0 time: 0:02:30 (avg: 0:02:30, remaining: 0:24:59)
Exported lines 100-199 to steam_app_data.csv. Batch 1 time: 0:02:34 (avg: 0:02:32, remaining: 0:22:45)
Exported lines 200-299 to steam_app_data.csv. Batch 2 time: 0:02:32 (avg: 0:02:32, remaining: 0:20:14)
Exported lines 300-399 to steam_app_data.csv. Batch 3 time: 0:02:33 (avg: 0:02:32, remaining: 0:17:45)
Exported lines 400-499 to steam_app_data.csv. Batch 4 time: 0:02:33 (avg: 0:02:32, remaining: 0:15:14)
Exported lines 500-599 to steam_app_data.csv. Batch 5 time: 0:02:33 (avg: 0:02:33, remaining: 0:12:43)
Exported lines 600-699 to steam_app_data.csv. Batch 6 time: 0:02:36 (avg: 0:02:33, remaining: 0:10:12)
Exported lines 700-799 to steam_app_data.csv. Batch 7 time: 0:02:32 (avg: 0:02:33, remaining: 0:07:39)
Exported lines 800-899 to steam_app_data.csv. Batch 8 time: 0:02:33 (avg: 0:02:33, remaining: 0:05:06)
Exported lines 900-999 to steam_app_data.csv. Batch 9 

In [ ]:
def parse_steamspy_request(appid, name):
    """Parser to handle SteamSpy API data."""
    url = "https://steamspy.com/api.php"
    parameters = {"request": "appdetails", "appid": appid}

    json_data = get_request(url, parameters)
    return json_data

download_path_spy = '/content'
steamspy_data = 'steamspy_data.csv'
steamspy_index = 'steamspy_index.txt'
steamspy_columns = [
    'appid', 'name', 'developer', 'publisher', 'score_rank', 'positive',
    'negative', 'userscore', 'owners', 'average_forever', 'average_2weeks',
    'median_forever', 'median_2weeks', 'price', 'initialprice', 'discount',
    'languages', 'genre', 'ccu', 'tags'
]

reset_index(download_path, steamspy_index)
index = get_index(download_path, steamspy_index)
prepare_data_file(download_path, steamspy_data, index, steamspy_columns)

process_batches(
    parser=parse_steamspy_request,
    app_list=app_list,
    download_path=download_path,
    data_filename=steamspy_data,
    index_filename=steamspy_index,
    columns=steamspy_columns,
    begin=index,
)

Starting at index 0:

Exported lines 0-99 to steamspy_data.csv. Batch 0 time: 0:02:13 (avg: 0:02:13, remaining: 0:22:10)
Exported lines 100-199 to steamspy_data.csv. Batch 1 time: 0:02:13 (avg: 0:02:13, remaining: 0:19:59)
Exported lines 200-299 to steamspy_data.csv. Batch 2 time: 0:02:13 (avg: 0:02:13, remaining: 0:17:45)
Exported lines 300-399 to steamspy_data.csv. Batch 3 time: 0:02:14 (avg: 0:02:13, remaining: 0:15:34)
Exported lines 400-499 to steamspy_data.csv. Batch 4 time: 0:02:15 (avg: 0:02:14, remaining: 0:13:22)
Exported lines 500-599 to steamspy_data.csv. Batch 5 time: 0:02:15 (avg: 0:02:14, remaining: 0:11:09)
Exported lines 600-699 to steamspy_data.csv. Batch 6 time: 0:02:14 (avg: 0:02:14, remaining: 0:08:56)
Exported lines 700-799 to steamspy_data.csv. Batch 7 time: 0:02:14 (avg: 0:02:14, remaining: 0:06:42)
Exported lines 800-899 to steamspy_data.csv. Batch 8 time: 0:02:15 (avg: 0:02:14, remaining: 0:04:28)
Exported lines 900-999 to steamspy_data.csv. Batch 9 time: 0:02



---


**DATA CLEANING STEAMSPY DATA**

---





In [ ]:
raw_steamspy_data = pd.read_csv('steamspy_data.csv')
raw_steamspy_data

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
0,1200,Red Orchestra: Ostfront 41-45,Tripwire Interactive,Tripwire Interactive,NaN,2042,300,0,"500,000 .. 1,000,000",0,0,0,0,324,499,35,"English, French, Russian",Action,15,"{'World War II': 161, 'Action': 127, 'FPS': 12..."
1,1510,Uplink,Introversion Software,Introversion Software,NaN,2063,198,0,"500,000 .. 1,000,000",0,0,0,0,199,999,80,English,"Indie, Strategy",3,"{'Hacking': 289, 'Indie': 216, 'Strategy': 202..."
2,1930,Two Worlds Epic Edition,Reality Pump Studios,Topware Interactive,NaN,4053,1272,0,"1,000,000 .. 2,000,000",0,0,0,0,399,999,60,"English, French, German, Italian, Spanish - Sp...",RPG,36,"{'RPG': 771, 'Open World': 191, 'Fantasy': 152..."
3,2270,Wolfenstein 3D,id Software,Bethesda Softworks,NaN,3194,210,0,"500,000 .. 1,000,000",0,0,0,0,149,499,70,English,Action,20,"{'FPS': 270, 'Action': 249, 'World War II': 24..."
4,2290,Final DOOM,id Software,id Software,NaN,1061,182,0,"500,000 .. 1,000,000",0,0,0,0,0,0,0,English,Action,2,"{'FPS': 128, 'Action': 109, 'Classic': 108, 'R..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2218750,Halls of Torment,Chasing Carrots,Chasing Carrots,NaN,17213,606,0,"500,000 .. 1,000,000",0,0,0,0,399,499,20,English,"Action, Indie, RPG, Early Access",1873,"{'Early Access': 586, 'Bullet Hell': 532, 'Dar..."
996,2231380,Tom Clancy's Ghost Recon Breakpoint,Ubisoft Paris,Ubisoft,NaN,8373,3674,0,"500,000 .. 1,000,000",0,0,0,0,1199,5999,80,"English, French, Italian, German, Spanish - Sp...","Action, Adventure",3818,"{'Action': 137, 'Multiplayer': 119, 'Open Worl..."
997,2231450,Pizza Tower,Tour De Pizza,Tour De Pizza,NaN,46801,653,0,"1,000,000 .. 2,000,000",0,0,0,0,1499,1999,25,English,"Action, Indie",691,"{'Great Soundtrack': 713, '2D Platformer': 668..."
998,2296990,We Were Here Expeditions: The FriendShip,Total Mayhem Games,Total Mayhem Games,NaN,15051,1271,0,"500,000 .. 1,000,000",0,0,0,0,399,399,0,"English, French, Italian, German, Spanish - Sp...","Adventure, Casual, Indie",304,"{'Co-op': 201, 'Puzzle': 188, 'Adventure': 176..."


In [ ]:
raw_steamspy_data.isnull().sum()

appid                0
name                 0
developer            3
publisher            1
score_rank         999
positive             0
negative             0
userscore            0
owners               0
average_forever      0
average_2weeks       0
median_forever       0
median_2weeks        0
price                0
initialprice         0
discount             0
languages            0
genre                3
ccu                  0
tags                 0
dtype: int64

In [ ]:
drop_cols = [
    'score_rank',
    'userscore',
    'genre', 'developer', 'publisher', 'price', 'initialprice', 'discount', ta
    'average_2weeks', 'median_2weeks', 'ccu'
]

In [ ]:
tags = raw_steamspy_data['tags']
print(tags[0])
tags.head()

{'World War II': 161, 'Action': 127, 'FPS': 125, 'Realistic': 97, 'Multiplayer': 89, 'Singleplayer': 88, 'Shooter': 75, 'War': 68, 'Tactical': 65, 'Military': 64, 'Historical': 45, 'Simulation': 45, 'Team-Based': 36, 'Classic': 30, 'Strategy': 23, 'Atmospheric': 22, 'First-Person': 13, 'Tanks': 11, 'Survival': 8}


0    {'World War II': 161, 'Action': 127, 'FPS': 12...
1    {'Hacking': 289, 'Indie': 216, 'Strategy': 202...
2    {'RPG': 771, 'Open World': 191, 'Fantasy': 152...
3    {'FPS': 270, 'Action': 249, 'World War II': 24...
4    {'FPS': 128, 'Action': 109, 'Classic': 108, 'R...
Name: tags, dtype: object

In [ ]:
values = [
    ['a', 'b'],
    ('b', 'c'),
    {'d': 'e'}
]

list(itertools.chain(*values))
set(itertools.chain(*values))
parsed_tags = tags.apply(lambda x: literal_eval(x))
cols = set(itertools.chain(*parsed_tags))

In [ ]:
def parse_tags(x):
    x = literal_eval(x)

    if isinstance(x, dict):
        return x
    elif isinstance(x, list):
        return {}
    else:
        raise TypeError('Something other than dict or list found')

parsed_tags = tags.apply(parse_tags)

tag_data = pd.DataFrame()

for col in sorted(cols):
    # standardise column names
    col_name = col.lower().replace(' ', '_').replace('-', '_').replace("'", "")
    tag_data[col_name] = parsed_tags.apply(lambda x: x[col] if col in x.keys() else 0)

tag_data.head()

<ipython-input-318-5dd3de270d9f>:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tag_data[col_name] = parsed_tags.apply(lambda x: x[col] if col in x.keys() else 0)
<ipython-input-318-5dd3de270d9f>:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tag_data[col_name] = parsed_tags.apply(lambda x: x[col] if col in x.keys() else 0)
<ipython-input-318-5dd3de270d9f>:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining a

,1980s,1990s,2.5d,2d,2d_fighter,2d_platformer,3d,3d_fighter,3d_platformer,3d_vision,4_player_local,4x,6dof,atv,abstract,action,action_rpg,action_rts,action_roguelike,action_adventure,addictive,adventure,agriculture,aliens,alternate_history,america,animation_&_modeling,anime,arcade,archery,arena_shooter,artificial_intelligence,assassin,asynchronous_multiplayer,atmospheric,audio_production,auto_battler,automation,automobile_sim,bmx,base_building,based_on_a_novel,basketball,battle_royale,beat_em_up,beautiful,benchmark,bikes,blood,board_game,boss_rush,bowling,boxing,building,bullet_hell,bullet_time,crpg,capitalism,card_battler,card_game,cartoon,cartoony,casual,cats,character_action_game,character_customization,chess,choices_matter,choose_your_own_adventure,cinematic,city_builder,class_based,classic,clicker,co_op,co_op_campaign,coding,cold_war,collectathon,colony_sim,colorful,combat,combat_racing,comedy,comic_book,competitive,conspiracy,controller,conversation,cooking,cozy,crafting,creature_collector,crime,crowdfunded,cult_classic,cute,cyberpunk,cycling,dark,dark_comedy,dark_fantasy,dark_humor,dating_sim,deckbuilding,demons,design_&_illustration,destruction,detective,difficult,dinosaurs,diplomacy,dog,dragons,drama,driving,dungeon_crawler,dungeons_&_dragons,dynamic_narration,dystopian_,early_access,economy,education,emotional,epic,episodic,escape_room,experience,experimental,exploration,extraction_shooter,fmv,fps,faith,family_friendly,fantasy,farming,farming_sim,fast_paced,female_protagonist,fighting,first_person,fishing,flight,football_(american),football_(soccer),foreign,free_to_play,funny,futuristic,gambling,game_development,gamemaker,games_workshop,gaming,god_game,golf,gore,gothic,grand_strategy,great_soundtrack,grid_based_movement,gun_customization,hack_and_slash,hacking,hand_drawn,hardware,heist,hentai,hero_shooter,hex_grid,hidden_object,historical,hockey,horror,horses,hunting,idler,illuminati,immersive,immersive_sim,indie,intentionally_awkward_controls,interactive_fiction,inventory_management,investigation,isometric,jrpg,jet,job_simulator,jump_scare,kickstarter,lego,lgbtq+,lemmings,level_editor,life_sim,linear,local_co_op,local_multiplayer,logic,loot,looter_shooter,lore_rich,lovecraftian,mmorpg,moba,magic,management,mars,martial_arts,massively_multiplayer,match_3,mature,mechs,medieval,memes,metroidvania,military,mini_golf,minigames,minimalist,mining,mod,moddable,modern,motocross,motorbike,mouse_only,movie,multiplayer,multiple_endings,music,music_based_procedural_generation,musou,mystery,mystery_dungeon,mythology,nsfw,narration,narrative,nature,naval,naval_combat,ninja,noir,nonlinear,nostalgia,nudity,offroad,old_school,online_co_op,open_world,open_world_survival_craft,parkour,parody_,party,party_game,party_based_rpg,perma_death,philosophical,photo_editing,physics,pinball,pirates,pixel_graphics,platformer,point_&_click,political,political_sim,politics,pool,post_apocalyptic,precision_platformer,procedural_generation,programming,psychedelic,psychological,psychological_horror,puzzle,puzzle_platformer,pve,pvp,quick_time_events,rpg,rpgmaker,rts,racing,real_time_tactics,real_time,real_time_with_pause,realistic,relaxing,remake,replay_value,resource_management,retro,rhythm,robots,rock_music,rogue_like,rogue_lite,roguelike_deckbuilder,romance,rome,runner,sailing,sandbox,satire,sci_fi,science,score_attack,sequel,sexual_content,shoot_em_up,shooter,short,side_scroller,silent_protagonist,simulation,singleplayer,skateboarding,skating,sniper,snow,snowboarding,social_deduction,software,software_training,sokoban,solitaire,souls_like,soundtrack,space,space_sim,spaceships,spectacle_fighter,spelling,split_screen,sports,stealth,steampunk,story_rich,strategy,strategy_rpg,stylized,submarine,superhero,supernatural,surreal,survival,survival_horror,swordplay,tabletop,tactical,tactical_rpg,tanks,team_based,text_based,third_person,third_person_shooter,thriller,time_attack,time_management,time_manipulation,time_travel,top_down,top_down_shooter,touch_friend

In [ ]:
def parse_tags(x):
    x = literal_eval(x)

    if isinstance(x, dict):
        return ';'.join(list(x.keys())[:3])
    else:
        return np.nan

tags.apply(parse_tags)

0                        World War II;Action;FPS
1                         Hacking;Indie;Strategy
2                         RPG;Open World;Fantasy
3                        FPS;Action;World War II
4                             FPS;Action;Classic
                         ...                    
995        Early Access;Bullet Hell;Dark Fantasy
996                Action;Multiplayer;Open World
997    Great Soundtrack;2D Platformer;Fast-Paced
998                       Co-op;Puzzle;Adventure
999             Early Access;Survival;Open World
Name: tags, Length: 1000, dtype: object

In [ ]:
owners = raw_steamspy_data['owners']
owners.head()

0      500,000 .. 1,000,000
1      500,000 .. 1,000,000
2    1,000,000 .. 2,000,000
3      500,000 .. 1,000,000
4      500,000 .. 1,000,000
Name: owners, dtype: object

In [ ]:
owners_split = owners.str.replace(',', '').str.split(' .. ')
owners_split.apply(lambda x: int(x[0])).head()

0     500000
1     500000
2    1000000
3     500000
4     500000
Name: owners, dtype: int64

In [ ]:
owners_split.apply(lambda x: (int(x[0]) + int(x[1])) // 2).head()

0     750000
1     750000
2    1500000
3     750000
4     750000
Name: owners, dtype: int64

In [ ]:
owners.str.replace(',', '').str.replace(' .. ', '-').head()

<ipython-input-323-e518ddcfc8ff>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  owners.str.replace(',', '').str.replace(' .. ', '-').head()


0     500000-1000000
1     500000-1000000
2    1000000-2000000
3     500000-1000000
4     500000-1000000
Name: owners, dtype: object

In [ ]:
def process_tags(df, export=False):
    if export:

        tag_data = df[['appid', 'tags']].copy()

        def parse_export_tags(x):
            x = literal_eval(x)

            if isinstance(x, dict):
                return x
            elif isinstance(x, list):
                return {}
            else:
                raise TypeError('Something other than dict or list found')

        tag_data['tags'] = tag_data['tags'].apply(parse_export_tags)

        cols = set(itertools.chain(*tag_data['tags']))

        for col in sorted(cols):
            col_name = col.lower().replace(' ', '_').replace('-', '_').replace("'", "")

            tag_data[col_name] = tag_data['tags'].apply(lambda x: x[col] if col in x.keys() else 0)

        tag_data = tag_data.drop('tags', axis=1)

        tag_data.to_csv('/content/steamspy_tag_data.csv', index=False)
        print("Exported tag data to 'steamspy_tag_data.csv'")
        files.download('/content/steamspy_tag_data.csv')


    def parse_tags(x):
        x = literal_eval(x)

        if isinstance(x, dict):
            return ';'.join(list(x.keys())[:3])
        else:
            return np.nan

    df['tags'] = df['tags'].apply(parse_tags)
    df = df[df['tags'].notnull()]
    return df


def process(df):
    df = df.copy()

    # handle missing values
    df = df[(df['name'].notnull()) & (df['name'] != 'none')]
    df = df[df['developer'].notnull()]
    df = df[df['languages'].notnull()]
    df = df[df['price'].notnull()]

    # remove unwanted columns
    df = df.drop([
        'genre', 'developer', 'publisher', 'score_rank', 'userscore', 'average_2weeks',
        'median_2weeks', 'price', 'initialprice', 'discount', 'ccu'
    ], axis=1)

    df = process_tags(df, export=True)
    df['owners'] = df['owners'].str.replace(',', '').str.replace(' .. ', '-')

    return df


steamspy_data = process(raw_steamspy_data)
steamspy_data.head()

<ipython-input-324-5dc34f2e72bc>:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tag_data[col_name] = tag_data['tags'].apply(lambda x: x[col] if col in x.keys() else 0)
<ipython-input-324-5dc34f2e72bc>:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tag_data[col_name] = tag_data['tags'].apply(lambda x: x[col] if col in x.keys() else 0)
<ipython-input-324-5dc34f2e72bc>:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider

Exported tag data to 'steamspy_tag_data.csv'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<ipython-input-324-5dc34f2e72bc>:67: FutureWarning: The default value of regex will change from True to False in a future version.
  df['owners'] = df['owners'].str.replace(',', '').str.replace(' .. ', '-')


,appid,name,positive,negative,owners,average_forever,median_forever,languages,tags
0,1200,Red Orchestra: Ostfront 41-45,2042,300,500000-1000000,0,0,"English, French, Russian",World War II;Action;FPS
1,1510,Uplink,2063,198,500000-1000000,0,0,English,Hacking;Indie;Strategy
2,1930,Two Worlds Epic Edition,4053,1272,1000000-2000000,0,0,"English, French, German, Italian, Spanish - Sp...",RPG;Open World;Fantasy
3,2270,Wolfenstein 3D,3194,210,500000-1000000,0,0,English,FPS;Action;World War II
4,2290,Final DOOM,1061,182,500000-1000000,0,0,English,FPS;Action;Classic


In [ ]:
pd.read_csv('steamspy_tag_data.csv').head()

,appid,1980s,1990s,2.5d,2d,2d_fighter,2d_platformer,3d,3d_fighter,3d_platformer,3d_vision,4_player_local,4x,6dof,atv,abstract,action,action_rpg,action_rts,action_roguelike,action_adventure,addictive,adventure,agriculture,aliens,alternate_history,america,animation_&_modeling,anime,arcade,archery,arena_shooter,artificial_intelligence,assassin,asynchronous_multiplayer,atmospheric,audio_production,auto_battler,automation,automobile_sim,bmx,base_building,based_on_a_novel,basketball,battle_royale,beat_em_up,beautiful,benchmark,bikes,blood,board_game,boss_rush,bowling,boxing,building,bullet_hell,bullet_time,crpg,capitalism,card_battler,card_game,cartoon,cartoony,casual,cats,character_action_game,character_customization,chess,choices_matter,choose_your_own_adventure,cinematic,city_builder,class_based,classic,clicker,co_op,co_op_campaign,coding,cold_war,collectathon,colony_sim,colorful,combat,combat_racing,comedy,comic_book,competitive,conspiracy,controller,conversation,cooking,cozy,crafting,creature_collector,crime,crowdfunded,cult_classic,cute,cyberpunk,cycling,dark,dark_comedy,dark_fantasy,dark_humor,dating_sim,deckbuilding,demons,design_&_illustration,destruction,detective,difficult,dinosaurs,diplomacy,dog,dragons,drama,driving,dungeon_crawler,dungeons_&_dragons,dynamic_narration,dystopian_,early_access,economy,education,emotional,epic,episodic,escape_room,experience,experimental,exploration,extraction_shooter,fmv,fps,faith,family_friendly,fantasy,farming,farming_sim,fast_paced,female_protagonist,fighting,first_person,fishing,flight,football_(american),football_(soccer),foreign,free_to_play,funny,futuristic,gambling,game_development,gamemaker,games_workshop,gaming,god_game,golf,gore,gothic,grand_strategy,great_soundtrack,grid_based_movement,gun_customization,hack_and_slash,hacking,hand_drawn,hardware,heist,hentai,hero_shooter,hex_grid,hidden_object,historical,hockey,horror,horses,hunting,idler,illuminati,immersive,immersive_sim,indie,intentionally_awkward_controls,interactive_fiction,inventory_management,investigation,isometric,jrpg,jet,job_simulator,jump_scare,kickstarter,lego,lgbtq+,lemmings,level_editor,life_sim,linear,local_co_op,local_multiplayer,logic,loot,looter_shooter,lore_rich,lovecraftian,mmorpg,moba,magic,management,mars,martial_arts,massively_multiplayer,match_3,mature,mechs,medieval,memes,metroidvania,military,mini_golf,minigames,minimalist,mining,mod,moddable,modern,motocross,motorbike,mouse_only,movie,multiplayer,multiple_endings,music,music_based_procedural_generation,musou,mystery,mystery_dungeon,mythology,nsfw,narration,narrative,nature,naval,naval_combat,ninja,noir,nonlinear,nostalgia,nudity,offroad,old_school,online_co_op,open_world,open_world_survival_craft,parkour,parody_,party,party_game,party_based_rpg,perma_death,philosophical,photo_editing,physics,pinball,pirates,pixel_graphics,platformer,point_&_click,political,political_sim,politics,pool,post_apocalyptic,precision_platformer,procedural_generation,programming,psychedelic,psychological,psychological_horror,puzzle,puzzle_platformer,pve,pvp,quick_time_events,rpg,rpgmaker,rts,racing,real_time_tactics,real_time,real_time_with_pause,realistic,relaxing,remake,replay_value,resource_management,retro,rhythm,robots,rock_music,rogue_like,rogue_lite,roguelike_deckbuilder,romance,rome,runner,sailing,sandbox,satire,sci_fi,science,score_attack,sequel,sexual_content,shoot_em_up,shooter,short,side_scroller,silent_protagonist,simulation,singleplayer,skateboarding,skating,sniper,snow,snowboarding,social_deduction,software,software_training,sokoban,solitaire,souls_like,soundtrack,space,space_sim,spaceships,spectacle_fighter,spelling,split_screen,sports,stealth,steampunk,story_rich,strategy,strategy_rpg,stylized,submarine,superhero,supernatural,surreal,survival,survival_horror,swordplay,tabletop,tactical,tactical_rpg,tanks,team_based,text_based,third_person,third_person_shooter,thriller,time_attack,time_management,time_manipulation,time_travel,top_down,top_down_shooter,touch_

In [ ]:
steamspy_data.isnull().sum()

appid              0
name               0
positive           0
negative           0
owners             0
average_forever    0
median_forever     0
languages          0
tags               0
dtype: int64

In [ ]:
steamspy_data.to_csv('/content/steamspy_clean.csv', index=False)

In [ ]:
pd.read_csv('/content/steamspy_clean.csv')

,appid,name,positive,negative,owners,average_forever,median_forever,languages,tags
0,1200,Red Orchestra: Ostfront 41-45,2042,300,500000-1000000,0,0,"English, French, Russian",World War II;Action;FPS
1,1510,Uplink,2063,198,500000-1000000,0,0,English,Hacking;Indie;Strategy
2,1930,Two Worlds Epic Edition,4053,1272,1000000-2000000,0,0,"English, French, German, Italian, Spanish - Sp...",RPG;Open World;Fantasy
3,2270,Wolfenstein 3D,3194,210,500000-1000000,0,0,English,FPS;Action;World War II
4,2290,Final DOOM,1061,182,500000-1000000,0,0,English,FPS;Action;Classic
...,...,...,...,...,...,...,...,...,...
992,2218750,Halls of Torment,17213,606,500000-1000000,0,0,English,Early Access;Bullet Hell;Dark Fantasy
993,2231380,Tom Clancy's Ghost Recon Breakpoint,8373,3674,500000-1000000,0,0,"English, French, Italian, German, Spanish - Sp...",Action;Multiplayer;Open World
994,2231450,Pizza Tower,46801,653,1000000-2000000,0,0,English,Great Soundtrack;2D Platformer;Fast-Paced
995,2296990,We Were Here Expeditions: The FriendShip,15051,1271,500000-1000000,0,0,"English, French, Italian, German, Spanish - Sp...",Co-op;Puzzle;Adventure




---


DATA CLEANING STEAM_APP_DATA

---



In [ ]:
raw_steam_data = pd.read_csv('steam_app_data.csv')
raw_steam_data

,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
0,game,Red Orchestra: Ostfront 41-45,1200,0,False,NaN,NaN,<h1>D-Day Anniversary 2016 Update for Darkest ...,Fight in the theatre of war that changed the w...,Fight in the theatre of war that changed the w...,NaN,"English, French, Russian",https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.redorchestragame.com/,{'minimum': '<strong>Minimum:</strong><br><ul ...,"{'minimum': '<ul class=""bb_ul""><li><strong>OS:...",{'minimum': '<strong>Minimum:</strong><br><ul ...,NaN,NaN,NaN,['Tripwire Interactive'],['Tripwire Interactive'],NaN,"{'currency': 'USD', 'initial': 499, 'final': 3...",[63],"[{'name': 'default', 'title': 'Buy Red Orchest...","{'windows': True, 'mac': True, 'linux': True}","{'score': 81, 'url': 'https://www.metacritic.c...",<strong>&quot;... RO is also one of the market...,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 1334},"{'total': 44, 'highlighted': [{'name': 'Kills ...","{'coming_soon': False, 'date': 'Mar 14, 2006'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
1,game,Uplink,1510,0,False,NaN,NaN,You play an Uplink Agent who makes a living by...,You play an Uplink Agent who makes a living by...,You play an Uplink Agent who makes a living by...,NaN,English,https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.uplink.co.uk/,[],[],[],NaN,NaN,NaN,['Introversion Software'],['Introversion Software'],NaN,"{'currency': 'USD', 'initial': 999, 'final': 1...","[112, 14002]","[{'name': 'default', 'title': 'Buy Uplink', 'd...","{'windows': True, 'mac': True, 'linux': True}","{'score': 75, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '23', 'description': 'Indie'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 1625},NaN,"{'coming_soon': False, 'date': 'Aug 23, 2006'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
2,game,Two Worlds Epic Edition,1930,17,False,NaN,"[986290, 301210, 637030]","... 300 years after Aziraal has been banished,...","... 300 years after Aziraal has been banished,...","... 300 years after Aziraal has been banished,...",NaN,"English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.2-worlds.com/,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,Copyright 1999-2023 by TopWare Interactive ACE...,NaN,NaN,['Reality Pump Studios'],['Topware Interactive'],NaN,"{'currency': 'USD', 'initial': 999, 'final': 3...","[1589, 18163]","[{'name': 'default', 'title': 'Buy Two Worlds ...","{'windows': True, 'mac': True, 'linux': True}","{'score': 65, 'url': 'https://www.metacritic.c...",“The big player alongside Oblivion and Gothic ...,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '3', 'description': 'RPG'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 996, 'name': 'Two Worlds Trailer', 'th...",{'total': 2337},NaN,"{'coming_soon': False, 'date': 'Apr 30, 2009'}","{'url': 'http://www.2-worlds.com/confirm.php',...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
3,game,Wolfenstein 3D,2270,17,False,NaN,NaN,"Developed by id Software in 1992, Wolfenstein ...","Developed by id Software in 1992, Wolfenstein ...",

In [ ]:
raw_steam_data.isnull().sum()

type                          2
name                          0
steam_appid                   0
required_age                  2
is_free                       2
controller_support          624
dlc                         508
detailed_description          2
about_the_game                2
short_description             2
fullgame                   1000
supported_languages           2
header_image                  2
website                     190
pc_requirements               2
mac_requirements              2
linux_requirements            2
legal_notice                407
drm_notice                  952
ext_user_account_notice     930
developers                    5
publishers                    2
demos                       908
price_overview              232
packages                    207
package_groups                2
platforms                     2
metacritic                  580
reviews                     648
categories                    7
genres                        3
screensh

In [ ]:
threshold = raw_steam_data.shape[0] // 2
print('Drop columns with more than {} missing rows'.format(threshold))
print()

drop_rows = raw_steam_data.columns[null_counts > threshold]

print('Columns to drop: {}'.format(list(drop_rows)))

Drop columns with more than 500 missing rows

Columns to drop: ['controller_support', 'dlc', 'fullgame', 'drm_notice', 'ext_user_account_notice', 'demos', 'metacritic', 'reviews']


In [ ]:
print('Rows to remove:', raw_steam_data[raw_steam_data['type'].isnull()].shape[0])

# preview rows with missing type data
raw_steam_data[raw_steam_data['type'].isnull()].head()

Rows to remove: 2


,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
61,NaN,Fallout: New Vegas,22490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
752,NaN,F1 2019,928600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
raw_steam_data['type'].value_counts(dropna=False)

game    998
NaN       2
Name: type, dtype: int64

In [ ]:
raw_steam_data[(raw_steam_data['name'].isnull()) | (raw_steam_data['name'] == 'none')]

,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors


In [ ]:
duplicate_rows = raw_steam_data[raw_steam_data.duplicated()]
print('Duplicate rows to remove:', duplicate_rows.shape[0])
duplicate_rows

Duplicate rows to remove: 0


,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors


In [ ]:
def drop_null_cols(df, thresh=0.5):
    """Drop columns with more than a certain proportion of missing values (Default 50%)."""
    cutoff_count = len(df) * thresh
    return df.dropna(thresh=cutoff_count, axis=1)


def process_name_type(df):
    """Remove null values in name and type columns, and remove type column."""
    df = df[df['type'].notnull()]

    df = df[df['name'].notnull()]
    df = df[df['name'] != 'none']

    df = df.drop('type', axis=1)

    return df


def process(df):
    """Process data set. Will eventually contain calls to all functions we write."""

    df = df.copy()
    df = df.drop_duplicates()
    df = drop_null_cols(df)
    df = process_name_type(df)

    return df

print(raw_steam_data.shape)
initial_processing = process(raw_steam_data)
print(initial_processing.shape)
initial_processing.head()

(1000, 39)
(998, 30)


,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,developers,publishers,price_overview,packages,package_groups,platforms,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
0,Red Orchestra: Ostfront 41-45,1200,0,False,<h1>D-Day Anniversary 2016 Update for Darkest ...,Fight in the theatre of war that changed the w...,Fight in the theatre of war that changed the w...,"English, French, Russian",https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.redorchestragame.com/,{'minimum': '<strong>Minimum:</strong><br><ul ...,"{'minimum': '<ul class=""bb_ul""><li><strong>OS:...",{'minimum': '<strong>Minimum:</strong><br><ul ...,NaN,['Tripwire Interactive'],['Tripwire Interactive'],"{'currency': 'USD', 'initial': 499, 'final': 3...",[63],"[{'name': 'default', 'title': 'Buy Red Orchest...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 1334},"{'total': 44, 'highlighted': [{'name': 'Kills ...","{'coming_soon': False, 'date': 'Mar 14, 2006'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
1,Uplink,1510,0,False,You play an Uplink Agent who makes a living by...,You play an Uplink Agent who makes a living by...,You play an Uplink Agent who makes a living by...,English,https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.uplink.co.uk/,[],[],[],NaN,['Introversion Software'],['Introversion Software'],"{'currency': 'USD', 'initial': 999, 'final': 1...","[112, 14002]","[{'name': 'default', 'title': 'Buy Uplink', 'd...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '23', 'description': 'Indie'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 1625},NaN,"{'coming_soon': False, 'date': 'Aug 23, 2006'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
2,Two Worlds Epic Edition,1930,17,False,"... 300 years after Aziraal has been banished,...","... 300 years after Aziraal has been banished,...","... 300 years after Aziraal has been banished,...","English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.2-worlds.com/,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,Copyright 1999-2023 by TopWare Interactive ACE...,['Reality Pump Studios'],['Topware Interactive'],"{'currency': 'USD', 'initial': 999, 'final': 3...","[1589, 18163]","[{'name': 'default', 'title': 'Buy Two Worlds ...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '3', 'description': 'RPG'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 996, 'name': 'Two Worlds Trailer', 'th...",{'total': 2337},NaN,"{'coming_soon': False, 'date': 'Apr 30, 2009'}","{'url': 'http://www.2-worlds.com/confirm.php',...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
3,Wolfenstein 3D,2270,17,False,"Developed by id Software in 1992, Wolfenstein ...","Developed by id Software in 1992, Wolfenstein ...",World War II rages. The Nazis are planning to ...,English,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '<strong>Minimum: </strong>A 100% ...,[],[],Wolfenstein 3D® and Wolfenstein 3D®: Spear of ...,['id Software'],['Bethesda Softworks'],"{'currency': 'USD', 'initial': 499, 'final': 1...","[416, 770867]","[{'name': 'default', 'title': 'Buy Wolfenstein...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","

In [ ]:
initial_processing['required_age'].value_counts(dropna=False).sort_index()

0      855
10       1
13      10
14       1
16       6
17     116
17+      1
18       8
Name: required_age, dtype: int64

In [ ]:
def process_age(df):
    """Format ratings in the age column to be in line with the PEGI Age Ratings system."""
    cut_points = [-1, 0, 3, 7, 12, 16, 2000]
    label_values = [0, 3, 7, 12, 16, 18]

    df['required_age'] = pd.to_numeric(df['required_age'], errors='coerce')
    df['required_age'] = pd.cut(df['required_age'], bins=cut_points, labels=label_values, include_lowest=True)
    return df

age_df = process_age(initial_processing)
age_df['required_age'].value_counts().sort_index()

0     855
3       0
7       0
12      1
16     17
18    124
Name: required_age, dtype: int64

In [ ]:
age_df['price_overview'].head()

0    {'currency': 'USD', 'initial': 499, 'final': 3...
1    {'currency': 'USD', 'initial': 999, 'final': 1...
2    {'currency': 'USD', 'initial': 999, 'final': 3...
3    {'currency': 'USD', 'initial': 499, 'final': 1...
4                                                  NaN
Name: price_overview, dtype: object

In [ ]:
age_df['platforms'].head()

0      {'windows': True, 'mac': True, 'linux': True}
1      {'windows': True, 'mac': True, 'linux': True}
2      {'windows': True, 'mac': True, 'linux': True}
3    {'windows': True, 'mac': False, 'linux': False}
4    {'windows': True, 'mac': False, 'linux': False}
Name: platforms, dtype: object

In [ ]:
platforms_first_row = age_df['platforms'].iloc[0]
print(type(platforms_first_row))
platforms_first_row

<class 'str'>


"{'windows': True, 'mac': True, 'linux': True}"

In [ ]:
eval_first_row = literal_eval(platforms_first_row)
print(type(eval_first_row))
print(eval_first_row)
eval_first_row['windows']

<class 'dict'>
{'windows': True, 'mac': True, 'linux': True}


True

In [ ]:
age_df['platforms'].isnull().sum()

0

In [ ]:
# create string of keys, joined on a semi-colon
';'.join(eval_first_row.keys())

'windows;mac;linux'

In [ ]:
platforms = {'windows': True, 'mac': True, 'linux': False}
print([x for x in platforms.keys() if platforms[x]])
';'.join(x for x in platforms.keys() if platforms[x])

['windows', 'mac']


'windows;mac'

In [ ]:
def process_platforms(df):
    """Split platforms column into separate boolean columns for each platform."""
    df = df.copy()
    def parse_platforms(x):
        d = literal_eval(x)
        return ';'.join(platform for platform in d.keys() if d[platform])
    df['platforms'] = df['platforms'].apply(parse_platforms)
    return df
platforms_df = process_platforms(age_df)
platforms_df['platforms'].value_counts()

windows              655
windows;mac;linux    189
windows;mac          125
windows;linux         29
Name: platforms, dtype: int64

In [ ]:
platforms_df['price_overview'].isnull().sum()

230

In [ ]:
free_and_null_price = platforms_df[(platforms_df['is_free']) & (platforms_df['price_overview'].isnull())]
free_and_null_price.shape[0]

161

In [ ]:
not_free_and_null_price = platforms_df[(platforms_df['is_free'] == False) & (platforms_df['price_overview'].isnull())]
not_free_and_null_price.head()

,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,developers,publishers,price_overview,packages,package_groups,platforms,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
4,Final DOOM,2290,0,False,"<strong>Two New, 32-Level DOOM II Episodes.</s...","<strong>Two New, 32-Level DOOM II Episodes.</s...","Two New, 32-Level DOOM II Episodes. Evilution:...",English,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '<strong>Minimum: </strong>A 100% ...,[],[],NaN,['id Software'],['id Software'],NaN,NaN,[],windows,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 1050},NaN,"{'coming_soon': False, 'date': 'Aug 3, 2007'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
8,RollerCoaster Tycoon® 3: Platinum,2700,0,False,Rollercoaster Tycoon 3 Platinum combines the e...,Rollercoaster Tycoon 3 Platinum combines the e...,Rollercoaster Tycoon 3 Platinum combines the e...,"English, French, Italian, German, Spanish - Sp...",https://cdn.akamai.steamstatic.com/steam/apps/...,https://www.frontier.co.uk/our-games/our-gameo...,{'minimum': '<strong>Minimum: </strong><br>\t\...,"{'minimum': '<ul class=""bb_ul""><li><strong>OS:...",[],"© 2004, 2005 Frontier Developments Ltd. All Ri...","['Frontier', 'Aspyr (Mac)']","['Frontier', 'Aspyr (Mac)']",NaN,NaN,[],windows,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '28', 'description': 'Simulation'}, {'...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 4012},NaN,"{'coming_soon': False, 'date': 'Mar 12, 2008'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
12,Rome: Total War™ - Alexander,4770,0,False,Rome: Total War - Alexander is the second offi...,Rome: Total War - Alexander is the second offi...,Rome: Total War - Alexander is the second offi...,English,https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.totalwar.com,{'minimum': '<strong>Minimum: </strong>Microso...,[],[],NaN,['The Creative Assembly'],['SEGA'],NaN,NaN,[],windows,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '2', 'description': 'Strategy'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 677},NaN,"{'coming_soon': False, 'date': 'Aug 28, 2007'}","{'url': 'https://support.sega.co.uk', 'email':...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
31,Grand Theft Auto,12170,0,False,Drive dozens of varied vehicles around three o...,Drive dozens of varied vehicles around three o...,Drive dozens of varied vehicles around three o...,English,https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.rockstargames.com/classics/gta.html,"{'minimum': '<ul class=""bb_ul""><li><strong>OS:...",[],[],NaN,['Rockstar North'],['Rockstar Games'],NaN,NaN,[],windows,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 233},NaN,"{'coming_soon': False, 'date': 'Jan 4, 2008'}","{'url': 'http://support.rockstargames.com/', '...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
32,Grand Theft Auto 2,12180,0,False,Grand Theft Auto is back. The cars are faster....,Grand Theft Auto is back. The cars are faster....,Grand Theft Auto is back. The cars are faster....,English,https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.rockstargames.com/classics/index.html,[],[],[],NaN,['Rockstar North'],['Rockstar Games'],NaN,NaN,[],windows,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 183

In [ ]:
def print_steam_links(df):
    """Print links to store page for apps in a dataframe."""
    url_base = "https://store.steampowered.com/app/"
    for i, row in df.iterrows():
        appid = row['steam_appid']
        name = row['name']
        print(name + ':', url_base + str(appid))
print_steam_links(not_free_and_null_price[:5])

Final DOOM: https://store.steampowered.com/app/2290
RollerCoaster Tycoon® 3: Platinum: https://store.steampowered.com/app/2700
Rome: Total War™ - Alexander: https://store.steampowered.com/app/4770
Grand Theft Auto: https://store.steampowered.com/app/12170
Grand Theft Auto 2: https://store.steampowered.com/app/12180


In [ ]:
def process_price(df):
    df = df.copy()

    def parse_price(x):
        if x is not np.nan:
            return literal_eval(x)
        else:
            return {'currency': 'GBP', 'initial': -1}

    df['price_overview'] = df['price_overview'].apply(parse_price)
    df['currency'] = df['price_overview'].apply(lambda x: x['currency'])
    df['price'] = df['price_overview'].apply(lambda x: x['initial'])

    # Set price of free games to 0
    df.loc[df['is_free'], 'price'] = 0

    return df

price_data = process_price(platforms_df)[['name', 'currency', 'price']]
price_data

,name,currency,price
0,Red Orchestra: Ostfront 41-45,USD,499
1,Uplink,USD,999
2,Two Worlds Epic Edition,USD,999
3,Wolfenstein 3D,USD,499
4,Final DOOM,GBP,-1
...,...,...,...
995,Halls of Torment,USD,499
996,Tom Clancy's Ghost Recon® Breakpoint,USD,5999
997,Pizza Tower,USD,1999
998,We Were Here Expeditions: The FriendShip,USD,399


In [ ]:
price_data[price_data['currency'] != 'GBP']

,name,currency,price
0,Red Orchestra: Ostfront 41-45,USD,499
1,Uplink,USD,999
2,Two Worlds Epic Edition,USD,999
3,Wolfenstein 3D,USD,499
5,DOOM II,USD,499
...,...,...,...
995,Halls of Torment,USD,499
996,Tom Clancy's Ghost Recon® Breakpoint,USD,5999
997,Pizza Tower,USD,1999
998,We Were Here Expeditions: The FriendShip,USD,399


In [ ]:
def process_price(df):
    """Process price_overview column into formatted price column."""
    df = df.copy()

    def parse_price(x):
        if x is not np.nan:
            return literal_eval(x)
        else:
            return {'currency': 'GBP', 'initial': -1}

    df['price_overview'] = df['price_overview'].apply(parse_price)
    df['currency'] = df['price_overview'].apply(lambda x: x['currency'])
    df['price'] = df['price_overview'].apply(lambda x: x['initial'])

    # set price of free games to 0
    df.loc[df['is_free'], 'price'] = 0
    df.loc[df['price'] > 0, 'price'] /= 100
    df = df.drop(['is_free','price_overview'], axis=1)

    return df


price_df = process_price(platforms_df)
price_df[['name', 'currency','price']]

,name,currency,price
0,Red Orchestra: Ostfront 41-45,USD,4.99
1,Uplink,USD,9.99
2,Two Worlds Epic Edition,USD,9.99
3,Wolfenstein 3D,USD,4.99
4,Final DOOM,GBP,-1.00
...,...,...,...
995,Halls of Torment,USD,4.99
996,Tom Clancy's Ghost Recon® Breakpoint,USD,59.99
997,Pizza Tower,USD,19.99
998,We Were Here Expeditions: The FriendShip,USD,3.99


In [ ]:
print(price_df[price_df['price'] == -1].shape[0])

69


In [ ]:
print('Null counts:', price_df['package_groups'].isnull().sum())
print('Empty list counts:', price_df[price_df['package_groups'] == "[]"].shape[0])

Null counts: 0
Empty list counts: 209


In [ ]:
missing_price_and_package = price_df[(price_df['price'] == -1) & (price_df['package_groups'] == "[]")]
print('Number of rows:', missing_price_and_package.shape[0], '\n')
print('First few rows:\n')
print_steam_links(missing_price_and_package[:5])
print('\nLast few rows:\n')
print_steam_links(missing_price_and_package[-10:-5])


Number of rows: 64 

First few rows:

Final DOOM: https://store.steampowered.com/app/2290
RollerCoaster Tycoon® 3: Platinum: https://store.steampowered.com/app/2700
Rome: Total War™ - Alexander: https://store.steampowered.com/app/4770
Grand Theft Auto: https://store.steampowered.com/app/12170
Grand Theft Auto 2: https://store.steampowered.com/app/12180

Last few rows:

F1® 2020: https://store.steampowered.com/app/1080110
New Frontier: https://store.steampowered.com/app/1104640
Totally Reliable Delivery Service Beta: https://store.steampowered.com/app/1106850
F1® 2021: https://store.steampowered.com/app/1134570
eFootball PES 2021 SEASON UPDATE: https://store.steampowered.com/app/1259970


In [ ]:
missing_price_have_package = price_df.loc[(price_df['price'] == -1) & (price_df['package_groups'] != "[]"), ['name', 'steam_appid', 'package_groups', 'price']]
print('Number of rows:', missing_price_have_package.shape[0], '\n')
print('First few rows:\n')
print_steam_links(missing_price_have_package[:5])
print('\nLast few rows:\n')
print_steam_links(missing_price_have_package[-10:-5])

Number of rows: 5 

First few rows:

Dragon Age: Origins: https://store.steampowered.com/app/17450
Red Faction Guerrilla Steam Edition: https://store.steampowered.com/app/20500
Rocksmith® 2014 Edition - Remastered: https://store.steampowered.com/app/221680
Viscera Cleanup Detail: Shadow Warrior: https://store.steampowered.com/app/255520
Crysis® 3: https://store.steampowered.com/app/1282690

Last few rows:



In [ ]:
def process_price(df):
    """Process price_overview column into formatted price column, and take care of package columns."""
    df = df.copy()

    def parse_price(x):
        if x is not np.nan:
            return literal_eval(x)
        else:
            return {'currency': 'GBP', 'initial': -1}

    df['price_overview'] = df['price_overview'].apply(parse_price)

    df['currency'] = df['price_overview'].apply(lambda x: x['currency'])
    df['price'] = df['price_overview'].apply(lambda x: x['initial'])

    df.loc[df['is_free'], 'price'] = 0
    df = df[df['price'] != -1]
    df.loc[df['is_free'], 'price'] = 0

    df.loc[df['price'] > 0, 'price'] /= 100

    # remove columns no longer needed
    df = df.drop(['is_free','price_overview'], axis=1)

    return df


price_df = process_price(platforms_df)
price_df

,name,steam_appid,required_age,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,developers,publishers,packages,package_groups,platforms,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors,currency,price
0,Red Orchestra: Ostfront 41-45,1200,0,<h1>D-Day Anniversary 2016 Update for Darkest ...,Fight in the theatre of war that changed the w...,Fight in the theatre of war that changed the w...,"English, French, Russian",https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.redorchestragame.com/,{'minimum': '<strong>Minimum:</strong><br><ul ...,"{'minimum': '<ul class=""bb_ul""><li><strong>OS:...",{'minimum': '<strong>Minimum:</strong><br><ul ...,NaN,['Tripwire Interactive'],['Tripwire Interactive'],[63],"[{'name': 'default', 'title': 'Buy Red Orchest...",windows;mac;linux,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 1334},"{'total': 44, 'highlighted': [{'name': 'Kills ...","{'coming_soon': False, 'date': 'Mar 14, 2006'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",USD,4.99
1,Uplink,1510,0,You play an Uplink Agent who makes a living by...,You play an Uplink Agent who makes a living by...,You play an Uplink Agent who makes a living by...,English,https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.uplink.co.uk/,[],[],[],NaN,['Introversion Software'],['Introversion Software'],"[112, 14002]","[{'name': 'default', 'title': 'Buy Uplink', 'd...",windows;mac;linux,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '23', 'description': 'Indie'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 1625},NaN,"{'coming_soon': False, 'date': 'Aug 23, 2006'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",USD,9.99
2,Two Worlds Epic Edition,1930,18,"... 300 years after Aziraal has been banished,...","... 300 years after Aziraal has been banished,...","... 300 years after Aziraal has been banished,...","English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.2-worlds.com/,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,Copyright 1999-2023 by TopWare Interactive ACE...,['Reality Pump Studios'],['Topware Interactive'],"[1589, 18163]","[{'name': 'default', 'title': 'Buy Two Worlds ...",windows;mac;linux,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '3', 'description': 'RPG'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 996, 'name': 'Two Worlds Trailer', 'th...",{'total': 2337},NaN,"{'coming_soon': False, 'date': 'Apr 30, 2009'}","{'url': 'http://www.2-worlds.com/confirm.php',...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",USD,9.99
3,Wolfenstein 3D,2270,18,"Developed by id Software in 1992, Wolfenstein ...","Developed by id Software in 1992, Wolfenstein ...",World War II rages. The Nazis are planning to ...,English,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '<strong>Minimum: </strong>A 100% ...,[],[],Wolfenstein 3D® and Wolfenstein 3D®: Spear of ...,['id Software'],['Bethesda Softworks'],"[416, 770867]","[{'name': 'default', 'title': 'Buy Wolfenstein...",windows,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 2940},NaN,"{'coming_soon': False, 'date': 'Aug 3, 2007'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",USD,4.99
5,DOOM II,2300,0,"Developed by id Software, and originally relea..

In [ ]:
price_df['supported_languages'].isnull().sum()
price_df[price_df['supported_languages'].isnull()]

,name,steam_appid,required_age,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,developers,publishers,platforms,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors,price


In [ ]:
print(price_df['supported_languages'])
price_df['supported_languages'].value_counts().head(10)

14              English, French, German, Spanish - Spain
25                                               English
28                                               English
35                                               English
47     English<strong>*</strong>, Russian, French, It...
                             ...                        
940    English, French, Spanish - Spain, Korean, Port...
941    English<strong>*</strong>, Simplified Chinese,...
942    English, Korean<strong>*</strong>, Japanese, T...
945    English<strong>*</strong>, French<strong>*</st...
948    Simplified Chinese<strong>*</strong><br><stron...
Name: supported_languages, Length: 161, dtype: object


English                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               29
English<strong>*</strong><br><strong>*</strong>languages with full audio support                                                                                                                                                                                                                                                               

In [ ]:
def process_language(df):
    """Process supported_languages column into a boolean 'is english' column."""
    df = df.copy()

    # drop rows with missing language data
    df = df.dropna(subset=['supported_languages'])
    df['english'] = df['supported_languages'].apply(lambda x: 1 if 'english' in x.lower() else 0)
    df = df.drop('supported_languages', axis=1)
    return df

language_df = process_language(price_df)
language_df[['name', 'english']].head()

,name,english
14,Bloodline Champions,1
25,Champions Online,1
28,Dragon Nest,1
35,America's Army 3,1
47,Zombie Panic! Source,1


In [ ]:
language_df['english'].value_counts()

1    160
0      1
Name: english, dtype: int64

In [ ]:
print('Developers null counts:', language_df['developers'].isnull().sum())
print('Developers empty list counts:', language_df[language_df['developers'] == "['']"].shape[0])
print('\nPublishers null counts:', language_df['publishers'].isnull().sum())
print('Publishers empty list counts:', language_df[language_df['publishers'] == "['']"].shape[0])

Developers null counts: 0
Developers empty list counts: 0

Publishers null counts: 0
Publishers empty list counts: 4


In [ ]:
no_dev = language_df[language_df['developers'].isnull()]

print('Total games missing developer:', no_dev.shape[0], '\n')

print_steam_links(no_dev[:5])

no_pub = language_df[language_df['publishers'] == "['']"]

print('\nTotal games missing publisher:', no_pub.shape[0], '\n')
print_steam_links(no_pub[:5])

no_dev_or_pub = language_df[(language_df['developers'].isnull()) & (language_df['publishers'] == "['']")]

print('\nTotal games missing developer and publisher:', no_dev_or_pub.shape[0], '\n')
print_steam_links(no_dev_or_pub[:5])

Total games missing developer: 0 


Total games missing publisher: 4 

Kingdom Wars: https://store.steampowered.com/app/227180
Hired Ops: https://store.steampowered.com/app/374280
Hero Zero - Multiplayer RPG: https://store.steampowered.com/app/482920
Drunken Wrestlers 2: https://store.steampowered.com/app/667530

Total games missing developer and publisher: 0 



In [ ]:
def process_developers_and_publishers(df):
    # remove rows with missing data
    df = df[(df['developers'].notnull()) & (df['publishers'] != "['']")].copy()

    for col in ['developers', 'publishers']:
        df[col] = df[col].apply(lambda x: literal_eval(x))

        # filter dataframe to rows with lists longer than 1, and store the number of rows
        num_rows = df[df[col].str.len() > 1].shape[0]

        print('Rows in {} column with multiple values:'.format(col), num_rows)

process_developers_and_publishers(language_df)

Rows in developers column with multiple values: 13
Rows in publishers column with multiple values: 5


In [ ]:
def process_developers_and_publishers(df):
    """Parse columns as semicolon-separated string."""
    # remove rows with missing data (~ means not)
    df = df[(df['developers'].notnull()) & (df['publishers'] != "['']")].copy()
    df = df[~(df['developers'].str.contains(';')) & ~(df['publishers'].str.contains(';'))]
    df = df[(df['publishers'] != "['NA']") & (df['publishers'] != "['N/A']")]

    # create list for each
    df['developer'] = df['developers'].apply(lambda x: ';'.join(literal_eval(x)))
    df['publisher'] = df['publishers'].apply(lambda x: ';'.join(literal_eval(x)))

    df = df.drop(['developers', 'publishers'], axis=1)

    return df

dev_pub_df = process_developers_and_publishers(language_df)
dev_pub_df[['name', 'steam_appid', 'developer', 'publisher']].head()

,name,steam_appid,developer,publisher
14,Bloodline Champions,6370,Stunlock Studios,Stunlock Studios
25,Champions Online,9880,Cryptic Studios,Gearbox Publishing
28,Dragon Nest,11610,Eyedentity Games Inc.,Eyedentity Games Inc.
35,America's Army 3,13140,U.S. Army,U.S. Army
47,Zombie Panic! Source,17500,Zombie Panic! Team,Zombie Panic! Team


In [ ]:
def process_categories_and_genres(df):
    df = df.copy()
    df = df[(df['categories'].notnull()) & (df['genres'].notnull())]

    for col in ['categories', 'genres']:
        df[col] = df[col].apply(lambda x: ';'.join(item['description'] for item in literal_eval(x)))

    return df

cat_gen_df = process_categories_and_genres(dev_pub_df)
cat_gen_df[['steam_appid', 'categories', 'genres']].head()

,steam_appid,categories,genres
14,6370,Single-player;Multi-player;MMO,Action;Free to Play;Massively Multiplayer
25,9880,Multi-player;MMO;Co-op;Steam Achievements,Free to Play;Massively Multiplayer;RPG
28,11610,Single-player;Multi-player;MMO;Co-op,Action;Free to Play;Massively Multiplayer;RPG
35,13140,Single-player;Multi-player;Steam Achievements;...,Action
47,17500,Multi-player;PvP;Online PvP;Co-op;Online Co-op...,Action;Casual;Free to Play;Indie


In [ ]:
print('Achievements null counts:', cat_gen_df['achievements'].isnull().sum())
print('Content Decsriptors null counts:', cat_gen_df['content_descriptors'].isnull().sum())

cat_gen_df[['name', 'achievements', 'content_descriptors']]

Achievements null counts: 72
Content Decsriptors null counts: 0


,name,achievements,content_descriptors
14,Bloodline Champions,NaN,"{'ids': [], 'notes': None}"
25,Champions Online,"{'total': 709, 'highlighted': [{'name': 'Canad...","{'ids': [], 'notes': None}"
28,Dragon Nest,{'total': 0},"{'ids': [], 'notes': None}"
35,America's Army 3,"{'total': 71, 'highlighted': [{'name': 'Deadey...","{'ids': [], 'notes': None}"
47,Zombie Panic! Source,"{'total': 106, 'highlighted': [{'name': 'Fight...","{'ids': [2, 5], 'notes': None}"
...,...,...,...
940,SuchArt: Creative Space,"{'total': 13, 'highlighted': [{'name': 'So, it...","{'ids': [], 'notes': None}"
941,A.V.A Global,"{'total': 6, 'highlighted': [{'name': 'MA-47 M...","{'ids': [2, 5], 'notes': 'The following list o..."
942,TheDawn,NaN,"{'ids': [], 'notes': None}"
945,IdleOn - The Idle MMO,"{'total': 61, 'highlighted': [{'name': 'The Sc...","{'ids': [], 'notes': None}"


In [ ]:
cat_gen_df['content_descriptors'].value_counts().head(6)

{'ids': [], 'notes': None}                                                                                                                                                                            137
{'ids': [2, 5], 'notes': None}                                                                                                                                                                          2
{'ids': [5], 'notes': 'Some female characters in the game are slightly exposed'}                                                                                                                        1
{'ids': [2, 5], 'notes': 'While the game itself features no violence or gore, there are trailers and gameplay videos for other games that players can view that contain gore and violent scenes.'}      1
{'ids': [5], 'notes': 'A small number of games included in the Ninja Kiwi archive contain graphic content and may not be suitable for younger players.'}                                        

In [ ]:
def process_achievements_and_descriptors(df):
    """Parse as total number of achievements."""
    df = df.copy()

    df = df.drop('content_descriptors', axis=1)

    def parse_achievements(x):
        if x is np.nan:
            # missing data, assume has no achievements
            return 0
        else:
            # else has data, so can extract and return number under total
            return literal_eval(x)['total']

    df['achievements'] = df['achievements'].apply(parse_achievements)

    return df

achiev_df = process_achievements_and_descriptors(cat_gen_df)
achiev_df['achievements'].value_counts().head()

0     77
64     4
14     3
24     3
11     3
Name: achievements, dtype: int64

In [ ]:
def process(df):
    """Process data set. Will eventually contain calls to all functions we write."""

    # Copy the input dataframe to avoid accidentally modifying original data
    df = df.copy()

    # Remove duplicate rows - all appids should be unique
    df = df.drop_duplicates()

    # Remove collumns with more than 50% null values
    df = drop_null_cols(df)

    # Process columns
    df = process_name_type(df)
    df = process_age(df)
    df = process_platforms(df)
    df = process_price(df)
    df = process_language(df)
    df = process_developers_and_publishers(df)
    df = process_categories_and_genres(df)
    df = process_achievements_and_descriptors(df)

    return df

partially_clean = process(raw_steam_data)
partially_clean

,name,steam_appid,required_age,detailed_description,about_the_game,short_description,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,platforms,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,price,english,developer,publisher
14,Bloodline Champions,6370,0,Engage in the most intense and fast paced play...,Engage in the most intense and fast paced play...,Bloodline Champions is a Free-to-Play online P...,https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.bloodlinechampions.com,"{'minimum': '<ul class=""bb_ul""><li><strong>Ope...",[],[],"Stunlock Studios, all rights reserved, Copyrig...",windows,Single-player;Multi-player;MMO,Action;Free to Play;Massively Multiplayer,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,NaN,0,"{'coming_soon': False, 'date': 'Oct 28, 2011'}",{'url': 'http://bloodlinechampions.com/support...,https://cdn.akamai.steamstatic.com/steam/apps/...,0.0,1,Stunlock Studios,Stunlock Studios
25,Champions Online,9880,0,"<h1>New Content</h1><p><img src=""https://cdn.a...",<strong>Champions Online</strong> brings epic ...,Grab your cape and defend Millennium City in t...,https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.arcgames.com/en/games/champions-online,{'minimum': '<strong>Minimum:</strong><br>\t\t...,[],[],"™ & © 2015 Cryptic Studios, Inc. All Rights Re...",windows,Multi-player;MMO;Co-op;Steam Achievements,Free to Play;Massively Multiplayer;RPG,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 180},709,"{'coming_soon': False, 'date': 'Jun 14, 2011'}","{'url': 'http://support.arcgames.com/', 'email...",https://cdn.akamai.steamstatic.com/steam/apps/...,0.0,1,Cryptic Studios,Gearbox Publishing
28,Dragon Nest,11610,0,Dragon Nest is an online action role-playing g...,Dragon Nest is an online action role-playing g...,Dragon Nest is an online action role-playing g...,https://cdn.akamai.steamstatic.com/steam/apps/...,http://us.dragonnest.com,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],© 2010 Eyedentity Games Inc. All Rights Reserved.,windows,Single-player;Multi-player;MMO;Co-op,Action;Free to Play;Massively Multiplayer;RPG,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 256670972, 'name': 'IceDragonNest_PV',...",NaN,0,"{'coming_soon': False, 'date': 'Dec 18, 2012'}",{'url': 'http://us.dragonnest.com/support/faq/...,https://cdn.akamai.steamstatic.com/steam/apps/...,0.0,1,Eyedentity Games Inc.,Eyedentity Games Inc.
35,America's Army 3,13140,0,"<i><h2 class=""bb_tag"">Check Out the latest ins...","<i><h2 class=""bb_tag"">Check Out the latest ins...",Download AA3 Today and Become a Member of the ...,https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.americasarmy.com,{'minimum': '<strong>Minimum:</strong> ...,[],[],"<img src=""https://cdn.akamai.steamstatic.com/s...",windows,Single-player;Multi-player;Steam Achievements;...,Action,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,NaN,71,"{'coming_soon': False, 'date': 'Jun 17, 2009'}","{'url': 'http://www.americasarmy.com/contact',...",https://cdn.akamai.steamstatic.com/steam/apps/...,0.0,1,U.S. Army,U.S. Army
47,Zombie Panic! Source,17500,0,<h1>Legacy 2.4 Build</h1><p>Are you looking fo...,Dive into the zombie apocalypse with Zombie Pa...,Dive into the zombie apocalypse with Zombie Pa...,https://cdn.akamai.steamstatic.com/steam/apps/...,https://zombiepanicsource.com,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],{'minimum': '<strong>Minimum:</strong><br><ul ...,NaN,windows;linux,Multi-player;PvP;Online PvP;Co-op;Online Co-op...,Action;Casual;Free to Play;Indie,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 256666956, 'name': 'Zombie Panic! Sour...",NaN,106,"{'coming_soon': False, 'date': 'Oct 3, 2008'}",{'url': 'https://forum.zombiepanicsource.com/'...,https://cdn.akamai.steamstatic.com/steam/apps/...,0.0,1,Zombie Panic! Team,Zombie Panic! Team
...,...,...,...,...,...,...,...,...

In [ ]:
partially_clean[['detailed_description', 'about_the_game', 'short_description']].isnull().sum()

detailed_description    0
about_the_game          0
short_description       0
dtype: int64

In [ ]:
partially_clean[partially_clean['detailed_description'].str.len() <= 100]

,name,steam_appid,required_age,detailed_description,about_the_game,short_description,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,platforms,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,price,english,developer,publisher


In [ ]:
def export_data(df, filename):
    """Export dataframe to csv file, filename prepended with 'steam_'.

    filename : str without file extension
    """
    filepath = '/content/steam_' + filename + '.csv'

    df.to_csv(filepath, index=False)

    print_name = filename.replace('_', ' ')
    print("Exported {} to '{}'".format(print_name, filepath))


In [ ]:
def process_descriptions(df, export=False):
    """Export descriptions to external csv file then remove these columns."""
    # remove rows with missing description data
    df = df[df['detailed_description'].notnull()].copy()

    # remove rows with unusually small description
    df = df[df['detailed_description'].str.len() > 20]

    # by default we don't export, useful if calling function later
    if export:
        # create dataframe of description columns
        description_data = df[['steam_appid', 'detailed_description', 'about_the_game', 'short_description']]

        export_data(description_data, filename='description_data')

    # drop description columns from main dataframe
    df = df.drop(['detailed_description', 'about_the_game', 'short_description'], axis=1)

    return df

desc_df = process_descriptions(partially_clean, export=True)

Exported description data to '/content/steam_description_data.csv'


In [ ]:
pd.read_csv('steam_description_data.csv').head()

,steam_appid,detailed_description,about_the_game,short_description
0,6370,Engage in the most intense and fast paced play...,Engage in the most intense and fast paced play...,Bloodline Champions is a Free-to-Play online P...
1,9880,"<h1>New Content</h1><p><img src=""https://cdn.a...",<strong>Champions Online</strong> brings epic ...,Grab your cape and defend Millennium City in t...
2,11610,Dragon Nest is an online action role-playing g...,Dragon Nest is an online action role-playing g...,Dragon Nest is an online action role-playing g...
3,13140,"<i><h2 class=""bb_tag"">Check Out the latest ins...","<i><h2 class=""bb_tag"">Check Out the latest ins...",Download AA3 Today and Become a Member of the ...
4,17500,<h1>Legacy 2.4 Build</h1><p>Are you looking fo...,Dive into the zombie apocalypse with Zombie Pa...,Dive into the zombie apocalypse with Zombie Pa...


In [ ]:
image_cols = ['header_image', 'screenshots', 'background']
for col in image_cols:
    print(col+':', desc_df[col].isnull().sum())
desc_df[image_cols].head()

header_image: 0
screenshots: 0
background: 0


,header_image,screenshots,background
14,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...
25,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...
28,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...
35,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...
47,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...


In [ ]:
no_screenshots = desc_df[desc_df['screenshots'].isnull()]
print_steam_links(no_screenshots)
no_screenshots

,name,steam_appid,required_age,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,platforms,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,price,english,developer,publisher


In [ ]:
print('Movies null values:', desc_df['movies'].isnull().sum())
print()
desc_df[desc_df['movies'].notnull()]['movies'].iloc[0]

Movies null values: 12



"[{'id': 256670972, 'name': 'IceDragonNest_PV', 'thumbnail': 'https://cdn.akamai.steamstatic.com/steam/apps/256670972/movie.293x165.jpg?t=1475032752', 'webm': {'480': 'http://cdn.akamai.steamstatic.com/steam/apps/256670972/movie480.webm?t=1475032752', 'max': 'http://cdn.akamai.steamstatic.com/steam/apps/256670972/movie_max.webm?t=1475032752'}, 'mp4': {'480': 'http://cdn.akamai.steamstatic.com/steam/apps/256670972/movie480.mp4?t=1475032752', 'max': 'http://cdn.akamai.steamstatic.com/steam/apps/256670972/movie_max.mp4?t=1475032752'}, 'highlight': True}, {'id': 256670973, 'name': 'Update_Wa&Ar&So_Arousal', 'thumbnail': 'https://cdn.akamai.steamstatic.com/steam/apps/256670973/movie.293x165.jpg?t=1475032879', 'webm': {'480': 'http://cdn.akamai.steamstatic.com/steam/apps/256670973/movie480.webm?t=1475032879', 'max': 'http://cdn.akamai.steamstatic.com/steam/apps/256670973/movie_max.webm?t=1475032879'}, 'mp4': {'480': 'http://cdn.akamai.steamstatic.com/steam/apps/256670973/movie480.mp4?t=14750

In [ ]:
def process_media(df, export=False):
    """Remove media columns from dataframe, optionally exporting them to csv first."""
    df = df[df['screenshots'].notnull()].copy()

    if export:
        media_data = df[['steam_appid', 'header_image', 'screenshots', 'background', 'movies']]

        export_data(media_data, 'media_data')

    df = df.drop(['header_image', 'screenshots', 'background', 'movies'], axis=1)

    return df

media_df = process_media(desc_df, export=True)

Exported media data to '/content/steam_media_data.csv'


In [ ]:
pd.read_csv('steam_media_data.csv').head()

,steam_appid,header_image,screenshots,background,movies
0,6370,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN
1,9880,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN
2,11610,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 256670972, 'name': 'IceDragonNest_PV',..."
3,13140,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN
4,17500,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 256666956, 'name': 'Zombie Panic! Sour..."


In [ ]:
achiev_df.info(verbose=False, memory_usage="deep")
media_df.info(verbose=False, memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 154 entries, 14 to 948
Columns: 26 entries, name to publisher
dtypes: category(1), float64(1), int64(3), object(21)
memory usage: 2.2 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 154 entries, 14 to 948
Columns: 19 entries, name to publisher
dtypes: category(1), float64(1), int64(3), object(14)
memory usage: 390.6 KB


In [ ]:
print('website null counts:', media_df['website'].isnull().sum())
print('support_info null counts:', media_df['support_info'].isnull().sum())

with pd.option_context("display.max_colwidth", 100): # ensures strings not cut short
    display(media_df[['name', 'website', 'support_info']][75:80])

website null counts: 24
support_info null counts: 0


,name,website,support_info
549,Eternal Card Game,https://www.EternalCardGame.com,"{'url': 'www.eternalcardgame.com', 'email': 'support@direwolfdigital.com'}"
550,MONMUSU,NaN,"{'url': '', 'email': 'info@amanitan.com'}"
551,Zombie Defense,http://www.homenetgames.com,"{'url': 'http://www.homenetgames.com/support/', 'email': 'support@homenetgames.com'}"
561,Darwin Project,https://www.play-darwinproject.com/,"{'url': '', 'email': 'support@scavengers.ca'}"
565,NosTale - Anime MMORPG,https://nostale.gameforge.com,"{'url': 'https://nostale.support.gameforge.com/en', 'email': ''}"


In [ ]:
def process_info(df, export=False):
    """Drop support information from dataframe, optionally exporting beforehand."""
    if export:
        support_info = df[['steam_appid', 'website', 'support_info']].copy()

        support_info['support_info'] = support_info['support_info'].apply(lambda x: literal_eval(x))
        support_info['support_url'] = support_info['support_info'].apply(lambda x: x['url'])
        support_info['support_email'] = support_info['support_info'].apply(lambda x: x['email'])

        support_info = support_info.drop('support_info', axis=1)

        # only keep rows with at least one piece of information
        support_info = support_info[(support_info['website'].notnull()) | (support_info['support_url'] != '') | (support_info['support_email'] != '')]

        export_data(support_info, 'support_info')

    df = df.drop(['website', 'support_info'], axis=1)

    return df


info_df = process_info(media_df, export=True)

Exported support info to '/content/steam_support_info.csv'


In [ ]:
pd.read_csv('steam_support_info.csv').head()

,steam_appid,website,support_url,support_email
0,6370,http://www.bloodlinechampions.com,http://bloodlinechampions.com/support/,support@stunlockstudios.com
1,9880,http://www.arcgames.com/en/games/champions-online,http://support.arcgames.com/,NaN
2,11610,http://us.dragonnest.com,http://us.dragonnest.com/support/faq/all,NaN
3,13140,http://www.americasarmy.com,http://www.americasarmy.com/contact,NaN
4,17500,https://zombiepanicsource.com,https://forum.zombiepanicsource.com/,support@zombiepanicsource.com


In [ ]:
requirements_cols = ['pc_requirements', 'mac_requirements', 'linux_requirements']
print('null counts:\n')
for col in requirements_cols:
    print(col+':', info_df[col].isnull().sum())


null counts:

pc_requirements: 0
mac_requirements: 0
linux_requirements: 0


In [ ]:
info_df[['steam_appid', 'pc_requirements', 'mac_requirements', 'linux_requirements']]

,steam_appid,pc_requirements,mac_requirements,linux_requirements
14,6370,"{'minimum': '<ul class=""bb_ul""><li><strong>Ope...",[],[]
25,9880,{'minimum': '<strong>Minimum:</strong><br>\t\t...,[],[]
28,11610,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[]
35,13140,{'minimum': '<strong>Minimum:</strong> ...,[],[]
47,17500,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],{'minimum': '<strong>Minimum:</strong><br><ul ...
...,...,...,...,...
940,1469280,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[]
941,1473480,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[]
942,1475380,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...
945,1476970,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[]


In [ ]:
print('Empty list counts:\n')
for col in requirements_cols:
    print(col+':', info_df[info_df[col] == '[]'].shape[0])

Empty list counts:

pc_requirements: 1
mac_requirements: 70
linux_requirements: 84


In [ ]:
for col in ['mac_requirements', 'linux_requirements']:
    platform = col.split('_')[0]
    print(platform+':', info_df[(info_df[col] == '[]') & (info_df['platforms'].str.contains(platform))].shape[0])

mac: 0
linux: 1


In [ ]:
print('windows:', info_df[(info_df['pc_requirements'] == '[]') & (info_df['platforms'].str.contains('windows'))].shape[0])

windows: 1


In [ ]:
missing_windows_requirements = info_df[(info_df['pc_requirements'] == '[]') & (info_df['platforms'].str.contains('windows'))]
print_steam_links(missing_windows_requirements)

Cat Burglar: A Tail of Purrsuit: https://store.steampowered.com/app/760510


In [ ]:
display(info_df['pc_requirements'].iloc[1])

'{\'minimum\': \'<strong>Minimum:</strong><br>\\t\\t\\t\\t\\t\\t\\t<ul class="bb_ul"><li><strong>OS:</strong> Windows 7<br>\\t\\t\\t\\t\\t\\t\\t</li><li><strong>Processor:</strong> 2.5GHz Single Core or 1.8GHz Dual Core<br>\\t\\t\\t\\t\\t\\t\\t</li><li><strong>Memory:</strong> 2GB RAM<br>\\t\\t\\t\\t\\t\\t\\t</li><li><strong>Graphics:</strong> NVIDIA GeForce 7800 / ATI Radeon X700 or HD / Intel Graphics with Dual Core<br>\\t\\t\\t\\t\\t\\t\\t</li><li><strong>DirectX®:</strong> DirectX 9.0c<br>\\t\\t\\t\\t\\t\\t\\t</li><li><strong>Hard Drive:</strong> 10GB Free Disk Space<br>\\t\\t\\t\\t\\t\\t\\t</li><li><strong>Sound:</strong> DirectX 9.0c Compatible Soundcard<br>\\t\\t\\t\\t\\t\\t\\t</li></ul>\', \'recommended\': \'<strong>Recommended:</strong><br>\\t\\t\\t\\t\\t\\t\\t<ul class="bb_ul"><li><strong>OS:</strong> Windows 7 / 8 / 10 64 bit<br>\\t\\t\\t\\t\\t\\t\\t</li><li><strong>Processor:</strong> 2.5GHz Dual Core or Better<br>\\t\\t\\t\\t\\t\\t\\t</li><li><strong>Memory:</strong> 3GB R

In [ ]:
view_requirements = info_df['pc_requirements'].iloc[[1]].copy()

view_requirements = (view_requirements
                         .str.replace(r'\\[rtn]', '')
                         .str.replace(r'<[pbr]{1,2}>', ' ')
                         .str.replace(r'<[\/"=\w\s]+>', '')
                    )

for i, row in view_requirements.iteritems():
    display(row)

<ipython-input-393-5be66452e087>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace(r'\\[rtn]', '')
<ipython-input-393-5be66452e087>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace(r'<[pbr]{1,2}>', ' ')
<ipython-input-393-5be66452e087>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace(r'<[\/"=\w\s]+>', '')
<ipython-input-393-5be66452e087>:9: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, row in view_requirements.iteritems():


"{'minimum': 'Minimum: OS: Windows 7 Processor: 2.5GHz Single Core or 1.8GHz Dual Core Memory: 2GB RAM Graphics: NVIDIA GeForce 7800 / ATI Radeon X700 or HD / Intel Graphics with Dual Core DirectX®: DirectX 9.0c Hard Drive: 10GB Free Disk Space Sound: DirectX 9.0c Compatible Soundcard ', 'recommended': 'Recommended: OS: Windows 7 / 8 / 10 64 bit Processor: 2.5GHz Dual Core or Better Memory: 3GB RAM or Better Graphics: NVIDIA GeForce 8800 or Better / ATI Radeon HD3850 or Better DirectX®: ®:   DirectX 9.0c Hard Drive: 10GB Free Disk Space Sound: DirectX 9.0c Compatible Soundcard '}"

In [ ]:
def process_requirements(df, export=False):
    if export:
        requirements = df[['steam_appid', 'pc_requirements', 'mac_requirements', 'linux_requirements']].copy()

        requirements = requirements[requirements['pc_requirements'] != '[]']

        requirements['requirements_clean'] = (requirements['pc_requirements']
                                                  .str.replace(r'\\[rtn]', '')
                                                  .str.replace(r'<[pbr]{1,2}>', ' ')
                                                  .str.replace(r'<[\/"=\w\s]+>', '')
                                             )

        requirements['requirements_clean'] = requirements['requirements_clean'].apply(lambda x: literal_eval(x))

        requirements['minimum'] = requirements['requirements_clean'].apply(lambda x: x['minimum'].replace('Minimum:', '').strip() if 'minimum' in x.keys() else np.nan)
        requirements['recommended'] = requirements['requirements_clean'].apply(lambda x: x['recommended'].replace('Recommended:', '').strip() if 'recommended' in x.keys() else np.nan)

        requirements = requirements.drop('requirements_clean', axis=1)

        export_data(requirements, 'requirements_data')

    df = df.drop(['pc_requirements', 'mac_requirements', 'linux_requirements'], axis=1)

    return df

reqs_df = process_requirements(info_df, export=True)

Exported requirements data to '/content/steam_requirements_data.csv'


<ipython-input-394-5601e74337af>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace(r'\\[rtn]', '')
<ipython-input-394-5601e74337af>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace(r'<[pbr]{1,2}>', ' ')
<ipython-input-394-5601e74337af>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace(r'<[\/"=\w\s]+>', '')


In [ ]:
pd.read_csv('steam_requirements_data.csv')

,steam_appid,pc_requirements,mac_requirements,linux_requirements,minimum,recommended
0,6370,"{'minimum': '<ul class=""bb_ul""><li><strong>Ope...",[],[],"Operating system:Windows® XP Service Pack 3, V...",NaN
1,9880,{'minimum': '<strong>Minimum:</strong><br>\t\t...,[],[],OS: Windows 7 Processor: 2.5GHz Single Core or...,OS: Windows 7 / 8 / 10 64 bit Processor: 2.5GH...
2,11610,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],"OS:Windows XP or higher Processor:Pentium 4, D...","OS:Windows XP or higher Processor:Pentium 4, D..."
3,13140,{'minimum': '<strong>Minimum:</strong> ...,[],[],Supported OS: Microsoft® Windows® XP SP2/Vista...,Supported OS: Microsoft® Windows® XP SP2/Vista...
4,17500,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],{'minimum': '<strong>Minimum:</strong><br><ul ...,OS: Windows® 7 (32/64-bits)/Vista Processor: 1...,OS: Windows® 7 (32/64-bits) or above Processor...
...,...,...,...,...,...,...
148,1469280,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],OS: 64-Bit Windows 7/8/10 Processor: Intel Cor...,NaN
149,1473480,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],OS: Windows7 64Bit Processor: Intel Core 2 Qua...,OS: Windows 10 64Bit Processor: Intel Core 2 Q...
150,1475380,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,Requires a 64-bit processor and operating syst...,Requires a 64-bit processor and operating syst...
151,1476970,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],OS: Windows 7+ Processor: An Intel Pentium 4 p...,NaN


In [ ]:
reqs_df.to_csv('/content/steam_partially_clean.csv', index=False)

In [ ]:

def process_release_date(df):
    df = df.copy()

    df['release_date'] = df['release_date'].apply(literal_eval)
    df = pd.concat([df.drop(['release_date'], axis=1), df['release_date'].apply(pd.Series)], axis=1)
    df.drop(columns=['coming_soon'], inplace=True)
    df['release_date'] = df['date'].apply(lambda x: x.replace(',', '') if pd.notnull(x) else np.nan)
    df['release_date'] = pd.to_datetime(df['release_date'], format='%b %d %Y', errors='coerce')
    df = df[df['release_date'].notnull()]

    df.drop(columns=['date'], inplace=True)

    return df



In [ ]:
def process(df):
    """Process data set. Will eventually contain calls to all functions we write."""

    df = df.copy()

    df = df.drop_duplicates()

    df = drop_null_cols(df)

    df = process_name_type(df)
    df = process_age(df)
    df = process_platforms(df)
    df = process_price(df)
    df = process_language(df)
    df = process_developers_and_publishers(df)
    df = process_categories_and_genres(df)
    df = process_achievements_and_descriptors(df)
    df = process_release_date(df)

    df = process_descriptions(df, export=True)
    df = process_media(df, export=True)
    df = process_info(df, export=True)
    df = process_requirements(df, export=True)

    return df

steam_data = process(raw_steam_data)
steam_data

Exported description data to '/content/steam_description_data.csv'
Exported media data to '/content/steam_media_data.csv'
Exported support info to '/content/steam_support_info.csv'
Exported requirements data to '/content/steam_requirements_data.csv'


<ipython-input-394-5601e74337af>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace(r'\\[rtn]', '')
<ipython-input-394-5601e74337af>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace(r'<[pbr]{1,2}>', ' ')
<ipython-input-394-5601e74337af>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace(r'<[\/"=\w\s]+>', '')


,name,steam_appid,required_age,legal_notice,packages,package_groups,platforms,categories,genres,recommendations,achievements,currency,price,english,developer,publisher,release_date
0,Red Orchestra: Ostfront 41-45,1200,0,NaN,[63],"[{'name': 'default', 'title': 'Buy Red Orchest...",windows;mac;linux,Multi-player;Steam Achievements;Valve Anti-Che...,Action,{'total': 1334},44,USD,4.99,1,Tripwire Interactive,Tripwire Interactive,2006-03-14
1,Uplink,1510,0,NaN,"[112, 14002]","[{'name': 'default', 'title': 'Buy Uplink', 'd...",windows;mac;linux,Single-player,Indie;Strategy,{'total': 1625},0,USD,9.99,1,Introversion Software,Introversion Software,2006-08-23
2,Two Worlds Epic Edition,1930,18,Copyright 1999-2023 by TopWare Interactive ACE...,"[1589, 18163]","[{'name': 'default', 'title': 'Buy Two Worlds ...",windows;mac;linux,Single-player;Multi-player;PvP;Online PvP;Co-o...,RPG,{'total': 2337},0,USD,9.99,1,Reality Pump Studios,Topware Interactive,2009-04-30
3,Wolfenstein 3D,2270,18,Wolfenstein 3D® and Wolfenstein 3D®: Spear of ...,"[416, 770867]","[{'name': 'default', 'title': 'Buy Wolfenstein...",windows,Single-player;Steam Cloud,Action,{'total': 2940},0,USD,4.99,1,id Software,Bethesda Softworks,2007-08-03
5,DOOM II,2300,0,"© 2022 Bethesda Softworks LLC, a ZeniMax Media...",[420],"[{'name': 'default', 'title': 'Buy DOOM II', '...",windows,Single-player;Multi-player;PvP;Shared/Split Sc...,Action,{'total': 6843},0,USD,4.99,1,id Software,id Software,2007-08-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Halls of Torment,2218750,0,NaN,[795112],"[{'name': 'default', 'title': 'Buy Halls of To...",windows;linux,Single-player;Steam Achievements;Full controll...,Action;Indie;RPG;Early Access,{'total': 17539},252,USD,4.99,1,Chasing Carrots,Chasing Carrots,2023-05-24
996,Tom Clancy's Ghost Recon® Breakpoint,2231380,18,© 2019 Ubisoft Entertainment. All Rights Reser...,"[799885, 799900, 799903, 799906]","[{'name': 'default', 'title': ""Buy Tom Clancy'...",windows,Single-player;Multi-player;Co-op;Online Co-op;...,Action;Adventure,{'total': 11271},0,USD,59.99,1,Ubisoft Paris,Ubisoft,2023-01-23
997,Pizza Tower,2231450,0,NaN,[799915],"[{'name': 'default', 'title': 'Buy Pizza Tower...",windows,Single-player;Steam Achievements;Full controll...,Action;Indie,{'total': 43666},74,USD,19.99,1,Tour De Pizza,Tour De Pizza,2023-01-26
998,We Were Here Expeditions: The FriendShip,2296990,0,© 2023 TMG Studios B.V. All rights reserved.,[946498],"[{'name': 'default', 'title': 'Buy We Were Her...",windows,Multi-player;Co-op;Online Co-op;Cross-Platform...,Adventure;Casual;Indie,{'total': 455},12,USD,3.99,1,Total Mayhem Games,Total Mayhem Games,2023-09-14


In [ ]:
steam_data.isnull().sum()
steam_data.drop(columns=['legal_notice', 'recommendations'], inplace=True)#Remove unneeded Data

In [ ]:
steam_data

,name,steam_appid,required_age,packages,package_groups,platforms,categories,genres,achievements,currency,price,english,developer,publisher,release_date
0,Red Orchestra: Ostfront 41-45,1200,0,[63],"[{'name': 'default', 'title': 'Buy Red Orchest...",windows;mac;linux,Multi-player;Steam Achievements;Valve Anti-Che...,Action,44,USD,4.99,1,Tripwire Interactive,Tripwire Interactive,2006-03-14
1,Uplink,1510,0,"[112, 14002]","[{'name': 'default', 'title': 'Buy Uplink', 'd...",windows;mac;linux,Single-player,Indie;Strategy,0,USD,9.99,1,Introversion Software,Introversion Software,2006-08-23
2,Two Worlds Epic Edition,1930,18,"[1589, 18163]","[{'name': 'default', 'title': 'Buy Two Worlds ...",windows;mac;linux,Single-player;Multi-player;PvP;Online PvP;Co-o...,RPG,0,USD,9.99,1,Reality Pump Studios,Topware Interactive,2009-04-30
3,Wolfenstein 3D,2270,18,"[416, 770867]","[{'name': 'default', 'title': 'Buy Wolfenstein...",windows,Single-player;Steam Cloud,Action,0,USD,4.99,1,id Software,Bethesda Softworks,2007-08-03
5,DOOM II,2300,0,[420],"[{'name': 'default', 'title': 'Buy DOOM II', '...",windows,Single-player;Multi-player;PvP;Shared/Split Sc...,Action,0,USD,4.99,1,id Software,id Software,2007-08-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Halls of Torment,2218750,0,[795112],"[{'name': 'default', 'title': 'Buy Halls of To...",windows;linux,Single-player;Steam Achievements;Full controll...,Action;Indie;RPG;Early Access,252,USD,4.99,1,Chasing Carrots,Chasing Carrots,2023-05-24
996,Tom Clancy's Ghost Recon® Breakpoint,2231380,18,"[799885, 799900, 799903, 799906]","[{'name': 'default', 'title': ""Buy Tom Clancy'...",windows,Single-player;Multi-player;Co-op;Online Co-op;...,Action;Adventure,0,USD,59.99,1,Ubisoft Paris,Ubisoft,2023-01-23
997,Pizza Tower,2231450,0,[799915],"[{'name': 'default', 'title': 'Buy Pizza Tower...",windows,Single-player;Steam Achievements;Full controll...,Action;Indie,74,USD,19.99,1,Tour De Pizza,Tour De Pizza,2023-01-26
998,We Were Here Expeditions: The FriendShip,2296990,0,[946498],"[{'name': 'default', 'title': 'Buy We Were Her...",windows,Multi-player;Co-op;Online Co-op;Cross-Platform...,Adventure;Casual;Indie,12,USD,3.99,1,Total Mayhem Games,Total Mayhem Games,2023-09-14


In [ ]:
raw_steam_data.info(verbose=False, memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Columns: 39 entries, type to content_descriptors
dtypes: float64(1), int64(1), object(37)
memory usage: 19.6 MB


In [ ]:
steam_data.info(verbose=False, memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 898 entries, 0 to 999
Columns: 15 entries, name to release_date
dtypes: category(1), datetime64[ns](1), float64(1), int64(3), object(9)
memory usage: 1.2 MB


In [ ]:
steam_data

,name,steam_appid,required_age,packages,package_groups,platforms,categories,genres,achievements,currency,price,english,developer,publisher,release_date
0,Red Orchestra: Ostfront 41-45,1200,0,[63],"[{'name': 'default', 'title': 'Buy Red Orchest...",windows;mac;linux,Multi-player;Steam Achievements;Valve Anti-Che...,Action,44,USD,4.99,1,Tripwire Interactive,Tripwire Interactive,2006-03-14
1,Uplink,1510,0,"[112, 14002]","[{'name': 'default', 'title': 'Buy Uplink', 'd...",windows;mac;linux,Single-player,Indie;Strategy,0,USD,9.99,1,Introversion Software,Introversion Software,2006-08-23
2,Two Worlds Epic Edition,1930,18,"[1589, 18163]","[{'name': 'default', 'title': 'Buy Two Worlds ...",windows;mac;linux,Single-player;Multi-player;PvP;Online PvP;Co-o...,RPG,0,USD,9.99,1,Reality Pump Studios,Topware Interactive,2009-04-30
3,Wolfenstein 3D,2270,18,"[416, 770867]","[{'name': 'default', 'title': 'Buy Wolfenstein...",windows,Single-player;Steam Cloud,Action,0,USD,4.99,1,id Software,Bethesda Softworks,2007-08-03
5,DOOM II,2300,0,[420],"[{'name': 'default', 'title': 'Buy DOOM II', '...",windows,Single-player;Multi-player;PvP;Shared/Split Sc...,Action,0,USD,4.99,1,id Software,id Software,2007-08-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Halls of Torment,2218750,0,[795112],"[{'name': 'default', 'title': 'Buy Halls of To...",windows;linux,Single-player;Steam Achievements;Full controll...,Action;Indie;RPG;Early Access,252,USD,4.99,1,Chasing Carrots,Chasing Carrots,2023-05-24
996,Tom Clancy's Ghost Recon® Breakpoint,2231380,18,"[799885, 799900, 799903, 799906]","[{'name': 'default', 'title': ""Buy Tom Clancy'...",windows,Single-player;Multi-player;Co-op;Online Co-op;...,Action;Adventure,0,USD,59.99,1,Ubisoft Paris,Ubisoft,2023-01-23
997,Pizza Tower,2231450,0,[799915],"[{'name': 'default', 'title': 'Buy Pizza Tower...",windows,Single-player;Steam Achievements;Full controll...,Action;Indie,74,USD,19.99,1,Tour De Pizza,Tour De Pizza,2023-01-26
998,We Were Here Expeditions: The FriendShip,2296990,0,[946498],"[{'name': 'default', 'title': 'Buy We Were Her...",windows,Multi-player;Co-op;Online Co-op;Cross-Platform...,Adventure;Casual;Indie,12,USD,3.99,1,Total Mayhem Games,Total Mayhem Games,2023-09-14


In [ ]:
steam_data.to_csv('/content/steam_data_clean.csv', index=False)



---


**DATA MERGING**

---



In [ ]:
steam_data = pd.read_csv('steam_data_clean.csv')
steam_data

,name,steam_appid,required_age,packages,package_groups,platforms,categories,genres,achievements,currency,price,english,developer,publisher,release_date
0,Red Orchestra: Ostfront 41-45,1200,0.0,[63],"[{'name': 'default', 'title': 'Buy Red Orchest...",windows;mac;linux,Multi-player;Steam Achievements;Valve Anti-Che...,Action,44,USD,4.99,1,Tripwire Interactive,Tripwire Interactive,2006-03-14
1,Uplink,1510,0.0,"[112, 14002]","[{'name': 'default', 'title': 'Buy Uplink', 'd...",windows;mac;linux,Single-player,Indie;Strategy,0,USD,9.99,1,Introversion Software,Introversion Software,2006-08-23
2,Two Worlds Epic Edition,1930,18.0,"[1589, 18163]","[{'name': 'default', 'title': 'Buy Two Worlds ...",windows;mac;linux,Single-player;Multi-player;PvP;Online PvP;Co-o...,RPG,0,USD,9.99,1,Reality Pump Studios,Topware Interactive,2009-04-30
3,Wolfenstein 3D,2270,18.0,"[416, 770867]","[{'name': 'default', 'title': 'Buy Wolfenstein...",windows,Single-player;Steam Cloud,Action,0,USD,4.99,1,id Software,Bethesda Softworks,2007-08-03
4,DOOM II,2300,0.0,[420],"[{'name': 'default', 'title': 'Buy DOOM II', '...",windows,Single-player;Multi-player;PvP;Shared/Split Sc...,Action,0,USD,4.99,1,id Software,id Software,2007-08-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
893,Halls of Torment,2218750,0.0,[795112],"[{'name': 'default', 'title': 'Buy Halls of To...",windows;linux,Single-player;Steam Achievements;Full controll...,Action;Indie;RPG;Early Access,252,USD,4.99,1,Chasing Carrots,Chasing Carrots,2023-05-24
894,Tom Clancy's Ghost Recon® Breakpoint,2231380,18.0,"[799885, 799900, 799903, 799906]","[{'name': 'default', 'title': ""Buy Tom Clancy'...",windows,Single-player;Multi-player;Co-op;Online Co-op;...,Action;Adventure,0,USD,59.99,1,Ubisoft Paris,Ubisoft,2023-01-23
895,Pizza Tower,2231450,0.0,[799915],"[{'name': 'default', 'title': 'Buy Pizza Tower...",windows,Single-player;Steam Achievements;Full controll...,Action;Indie,74,USD,19.99,1,Tour De Pizza,Tour De Pizza,2023-01-26
896,We Were Here Expeditions: The FriendShip,2296990,0.0,[946498],"[{'name': 'default', 'title': 'Buy We Were Her...",windows,Multi-player;Co-op;Online Co-op;Cross-Platform...,Adventure;Casual;Indie,12,USD,3.99,1,Total Mayhem Games,Total Mayhem Games,2023-09-14


In [ ]:
steamspy_data = pd.read_csv('steamspy_clean.csv')
steamspy_data

,appid,name,positive,negative,owners,average_forever,median_forever,languages,tags
0,1200,Red Orchestra: Ostfront 41-45,2042,300,500000-1000000,0,0,"English, French, Russian",World War II;Action;FPS
1,1510,Uplink,2063,198,500000-1000000,0,0,English,Hacking;Indie;Strategy
2,1930,Two Worlds Epic Edition,4053,1272,1000000-2000000,0,0,"English, French, German, Italian, Spanish - Sp...",RPG;Open World;Fantasy
3,2270,Wolfenstein 3D,3194,210,500000-1000000,0,0,English,FPS;Action;World War II
4,2290,Final DOOM,1061,182,500000-1000000,0,0,English,FPS;Action;Classic
...,...,...,...,...,...,...,...,...,...
992,2218750,Halls of Torment,17213,606,500000-1000000,0,0,English,Early Access;Bullet Hell;Dark Fantasy
993,2231380,Tom Clancy's Ghost Recon Breakpoint,8373,3674,500000-1000000,0,0,"English, French, Italian, German, Spanish - Sp...",Action;Multiplayer;Open World
994,2231450,Pizza Tower,46801,653,1000000-2000000,0,0,English,Great Soundtrack;2D Platformer;Fast-Paced
995,2296990,We Were Here Expeditions: The FriendShip,15051,1271,500000-1000000,0,0,"English, French, Italian, German, Spanish - Sp...",Co-op;Puzzle;Adventure


In [ ]:
merged = steam_data.merge(steamspy_data, left_on='steam_appid', right_on='appid', suffixes=('', '_steamspy'))
merged

,name,steam_appid,required_age,packages,package_groups,platforms,categories,genres,achievements,currency,price,english,developer,publisher,release_date,appid,name_steamspy,positive,negative,owners,average_forever,median_forever,languages,tags
0,Red Orchestra: Ostfront 41-45,1200,0.0,[63],"[{'name': 'default', 'title': 'Buy Red Orchest...",windows;mac;linux,Multi-player;Steam Achievements;Valve Anti-Che...,Action,44,USD,4.99,1,Tripwire Interactive,Tripwire Interactive,2006-03-14,1200,Red Orchestra: Ostfront 41-45,2042,300,500000-1000000,0,0,"English, French, Russian",World War II;Action;FPS
1,Uplink,1510,0.0,"[112, 14002]","[{'name': 'default', 'title': 'Buy Uplink', 'd...",windows;mac;linux,Single-player,Indie;Strategy,0,USD,9.99,1,Introversion Software,Introversion Software,2006-08-23,1510,Uplink,2063,198,500000-1000000,0,0,English,Hacking;Indie;Strategy
2,Two Worlds Epic Edition,1930,18.0,"[1589, 18163]","[{'name': 'default', 'title': 'Buy Two Worlds ...",windows;mac;linux,Single-player;Multi-player;PvP;Online PvP;Co-o...,RPG,0,USD,9.99,1,Reality Pump Studios,Topware Interactive,2009-04-30,1930,Two Worlds Epic Edition,4053,1272,1000000-2000000,0,0,"English, French, German, Italian, Spanish - Sp...",RPG;Open World;Fantasy
3,Wolfenstein 3D,2270,18.0,"[416, 770867]","[{'name': 'default', 'title': 'Buy Wolfenstein...",windows,Single-player;Steam Cloud,Action,0,USD,4.99,1,id Software,Bethesda Softworks,2007-08-03,2270,Wolfenstein 3D,3194,210,500000-1000000,0,0,English,FPS;Action;World War II
4,DOOM II,2300,0.0,[420],"[{'name': 'default', 'title': 'Buy DOOM II', '...",windows,Single-player;Multi-player;PvP;Shared/Split Sc...,Action,0,USD,4.99,1,id Software,id Software,2007-08-03,2300,DOOM II,7512,378,500000-1000000,0,0,"English, French, Italian, German, Spanish - Spain",Classic;FPS;Action
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892,Halls of Torment,2218750,0.0,[795112],"[{'name': 'default', 'title': 'Buy Halls of To...",windows;linux,Single-player;Steam Achievements;Full controll...,Action;Indie;RPG;Early Access,252,USD,4.99,1,Chasing Carrots,Chasing Carrots,2023-05-24,2218750,Halls of Torment,17213,606,500000-1000000,0,0,English,Early Access;Bullet Hell;Dark Fantasy
893,Tom Clancy's Ghost Recon® Breakpoint,2231380,18.0,"[799885, 799900, 799903, 799906]","[{'name': 'default', 'title': ""Buy Tom Clancy'...",windows,Single-player;Multi-player;Co-op;Online Co-op;...,Action;Adventure,0,USD,59.99,1,Ubisoft Paris,Ubisoft,2023-01-23,2231380,Tom Clancy's Ghost Recon Breakpoint,8373,3674,500000-1000000,0,0,"English, French, Italian, German, Spanish - Sp...",Action;Multiplayer;Open World
894,Pizza Tower,2231450,0.0,[799915],"[{'name': 'default', 'title': 'Buy Pizza Tower...",windows,Single-player;Steam Achievements;Full controll...,Action;Indie,74,USD,19.99,1,Tour De Pizza,Tour De Pizza,2023-01-26,2231450,Pizza Tower,46801,653,1000000-2000000,0,0,English,Great Soundtrack;2D Platformer;Fast-Paced
895,We Were Here Expeditions: The FriendShip,2296990,0.0,[946498],"[{'name': 'default', 'title': 'Buy We Were Her...",windows,Multi-player;Co-op;Online Co-op;Cross-Platform...,Adventure;Casual;Indie,12,USD,3.99,1,Total Mayhem Games,Total Mayhem Games,2023-09-14,2296990,We Were Here Expeditions: The FriendShip,15051,1271,500000-1000000,0,0,"English, French, Italian, German, Spanish - Sp...",Co-op;Puzzle;Adventure


In [ ]:
steam_clean = merged.drop(['name_steamspy', 'languages', 'steam_appid'], axis=1)
steam_clean = steam_clean[[
    'appid',
    'name',
    'release_date',
    'english',
    'developer',
    'publisher',
    'platforms',
    'required_age',
    'categories',
    'genres',
    'tags',
    'achievements',
    'positive',
    'negative',
    'average_forever',
    'median_forever',
    'owners',
    'currency','price'
]]

steam_clean = steam_clean.rename({
    'tags': 'steamspy_tags',
    'positive': 'positive_ratings',
    'negative': 'negative_ratings',
    'average_forever': 'average_playtime',
    'median_forever': 'median_playtime'
}, axis=1)

steam_clean

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,currency,price
0,1200,Red Orchestra: Ostfront 41-45,2006-03-14,1,Tripwire Interactive,Tripwire Interactive,windows;mac;linux,0.0,Multi-player;Steam Achievements;Valve Anti-Che...,Action,World War II;Action;FPS,44,2042,300,0,0,500000-1000000,USD,4.99
1,1510,Uplink,2006-08-23,1,Introversion Software,Introversion Software,windows;mac;linux,0.0,Single-player,Indie;Strategy,Hacking;Indie;Strategy,0,2063,198,0,0,500000-1000000,USD,9.99
2,1930,Two Worlds Epic Edition,2009-04-30,1,Reality Pump Studios,Topware Interactive,windows;mac;linux,18.0,Single-player;Multi-player;PvP;Online PvP;Co-o...,RPG,RPG;Open World;Fantasy,0,4053,1272,0,0,1000000-2000000,USD,9.99
3,2270,Wolfenstein 3D,2007-08-03,1,id Software,Bethesda Softworks,windows,18.0,Single-player;Steam Cloud,Action,FPS;Action;World War II,0,3194,210,0,0,500000-1000000,USD,4.99
4,2300,DOOM II,2007-08-03,1,id Software,id Software,windows,0.0,Single-player;Multi-player;PvP;Shared/Split Sc...,Action,Classic;FPS;Action,0,7512,378,0,0,500000-1000000,USD,4.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892,2218750,Halls of Torment,2023-05-24,1,Chasing Carrots,Chasing Carrots,windows;linux,0.0,Single-player;Steam Achievements;Full controll...,Action;Indie;RPG;Early Access,Early Access;Bullet Hell;Dark Fantasy,252,17213,606,0,0,500000-1000000,USD,4.99
893,2231380,Tom Clancy's Ghost Recon® Breakpoint,2023-01-23,1,Ubisoft Paris,Ubisoft,windows,18.0,Single-player;Multi-player;Co-op;Online Co-op;...,Action;Adventure,Action;Multiplayer;Open World,0,8373,3674,0,0,500000-1000000,USD,59.99
894,2231450,Pizza Tower,2023-01-26,1,Tour De Pizza,Tour De Pizza,windows,0.0,Single-player;Steam Achievements;Full controll...,Action;Indie,Great Soundtrack;2D Platformer;Fast-Paced,74,46801,653,0,0,1000000-2000000,USD,19.99
895,2296990,We Were Here Expeditions: The FriendShip,2023-09-14,1,Total Mayhem Games,Total Mayhem Games,windows,0.0,Multi-player;Co-op;Online Co-op;Cross-Platform...,Adventure;Casual;Indie,Co-op;Puzzle;Adventure,12,15051,1271,0,0,500000-1000000,USD,3.99


In [ ]:
steam_clean.to_csv('/content/steam_clean.csv', index=False)